In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

# Which Months

In [4]:
Monthsinc= '04_20-03_23'

# Amazon Subs (Importing, Merging, Transforming)

In [5]:
SFilepath='C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/SVOD/Amazon/Amazon Subs/'

Channels = ['Screambox', 'Docurama', 'CONtv', 'Dove Channel']
CountryF = ['US','UK']

In [6]:
#importing Subs for all subs except Fandor

lists = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022','2022-08':'2022','2022-09':'2022','2022-10':'2022','2022-11':'2022','2022-12':'2022',
         '2023-01':'2023','2023-02':'2023','2023-03':'2023'}
l=[]

for i in Channels: 
    for key,value in lists.items():
        l = l + [SFilepath + i +'/' + value + '/subs_m_US_' + i + '_' + key + '.csv']
        

In [7]:
#importing Subs for Fandor (Since UK data is always late)

listsF = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022','2022-08':'2022','2022-09':'2022','2022-10':'2022','2022-11':'2022','2022-12':'2022',
         '2023-01':'2023','2023-02':'2023','2023-03':'2023'}
lF=[]

for i in CountryF: 
    for key,value in listsF.items():
        lF = lF + [SFilepath + 'Fandor/' + i +'/' + value + '/subs_m_' + i + '_' + 'Fandor' + '_' + key + '.csv']
        

In [8]:
dfsubs = pd.concat([pd.read_csv(file) for file in l]).reset_index()
dfsubs= dfsubs.loc[dfsubs['Subscription Category']== 'HO',('Report Start','Subscription','Billing Amount','Active Subscriptions')]

In [9]:
dfFsubs = pd.concat([pd.read_csv(file) for file in lF]).reset_index()
# Fandor_last= pd.read_csv(SFilepath + 'Fandor/US/2022/subs_m_US_Fandor_2022-04.csv')
# dfFsubs = pd.concat([dfFsubs, Fandor_last])

dfFsubs= dfFsubs.loc[dfFsubs['Subscription Category']== 'HO',('Report Start','Subscription','Billing Amount','Active Subscriptions')]
dfFsubs['Subscription']='Fandor'
dfFsubs= dfFsubs.groupby(by=['Report Start','Subscription']).agg(sum).reset_index()
dfFsubs['Billing Amount']= 3.99

In [10]:
Amz_subs= pd.concat([dfsubs,dfFsubs]).reset_index()
Amz_subs.rename(columns={'Active Subscriptions':'Amazon Paid Subs'}, inplace=True)
Amz_subs['Report Start']= pd.to_datetime(Amz_subs['Report Start'])
Amz_subs.loc[Amz_subs['Subscription']=='Dove Channel', 'Subscription'] = 'Dove'
Amz_subs.loc[Amz_subs['Billing Amount']==2.99,'Billing Amount']=4.99

In [11]:
Amz_subs = Amz_subs.loc[:,('Subscription','Report Start','Billing Amount','Amazon Paid Subs')].groupby(by=['Subscription','Report Start','Billing Amount']).agg(sum).reset_index()

In [12]:
Amz_subs.head()

,Subscription,Report Start,Billing Amount,Amazon Paid Subs
0,CONtv,2020-04-01,4.99,26140
1,CONtv,2020-05-01,4.99,27697
2,CONtv,2020-06-01,4.99,32540
3,CONtv,2020-07-01,4.99,34989
4,CONtv,2020-08-01,4.99,35142


# Roku Subs (Importing, Transforming)

In [13]:
RokuFilepath='C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/SVOD/Roku/Roku Subs/'

In [14]:
CONtv = pd.read_excel(RokuFilepath + 'CONtv_Roku_SVOD_Subs.xlsx')
Docurama = pd.read_excel(RokuFilepath + 'Docurama_Roku_SVOD_Subs.xlsx')
Dove = pd.read_excel(RokuFilepath + 'Dove_Roku_SVOD_Subs.xlsx')
Screambox = pd.read_excel(RokuFilepath + 'Screambox_Roku_SVOD_Subs.xlsx')
Fandor = pd.read_excel(RokuFilepath + 'Fandor_Roku_SVOD_Subs.xlsx')

In [15]:
Rok_subs = pd.concat([CONtv, Docurama, Dove, Screambox, Fandor])

In [16]:
Rok_subs['Date Key']=pd.to_datetime(Rok_subs['Date Key'])
Rok_subs.rename(columns={'Date Key':'Report Start','Provider Product ID':'Subscription','Total Active Paid End Users':'Roku Paid Subs'}, inplace=True)

In [17]:
Rok_subs.head()

,Report Start,Subscription,Total New Paid End Users,Total New Free Trial End Users,Total Canceled Users,Roku Paid Subs,Total Active end Users,Total Active Free Trial End Users,Total Service Viewers,Trial End Conversion Rate,Churn Rate,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,2020-04-01,CONtv,157,3586,1394,1208,3727,2519,3897,0.536,0.180,NaN,NaN,NaN,NaN
1,2020-05-01,CONtv,1493,2713,2789,2279,3683,1404,2939,0.327,0.257,NaN,NaN,NaN,NaN
2,2020-06-01,CONtv,993,714,1698,2613,2757,144,1542,0.365,0.267,NaN,NaN,NaN,NaN
3,2020-07-01,CONtv,409,713,956,2431,2590,159,1272,0.472,0.230,NaN,NaN,NaN,NaN
4,2020-08-01,CONtv,315,503,786,2232,2314,82,781,0.493,0.220,NaN,NaN,NaN,NaN


# Merge Subs into one Table 'Subs'

In [18]:
Subs= pd.merge(Amz_subs, Rok_subs.loc[:,('Report Start','Subscription','Roku Paid Subs')], on= ['Report Start', 'Subscription'], how= 'left')

In [19]:
Subs['Year']= Subs['Report Start'].dt.year
Subs['Month(num)']= Subs['Report Start'].dt.month
Subs['Total Paid Subs']=Subs['Amazon Paid Subs']+ Subs['Roku Paid Subs']
Subs['Revenue']= Subs['Billing Amount']* Subs['Total Paid Subs']

In [20]:
Cond=[
    Subs['Month(num)']==1, Subs['Month(num)']==2, Subs['Month(num)']==3, Subs['Month(num)']==4,
    Subs['Month(num)']==5, Subs['Month(num)']==6, Subs['Month(num)']==7, Subs['Month(num)']==8,
    Subs['Month(num)']==9, Subs['Month(num)']==10, Subs['Month(num)']==11, Subs['Month(num)']==12
]
Months_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Subs['Month'] = np.select(Cond,Months_names, default=np.nan)

In [21]:
Subs=Subs.loc[:,('Year','Month(num)','Month','Subscription','Amazon Paid Subs','Roku Paid Subs','Billing Amount','Total Paid Subs','Revenue')]
Subs= Subs.sort_values(by=['Subscription','Year','Month(num)'])

In [22]:
Subs.head(10)

,Year,Month(num),Month,Subscription,Amazon Paid Subs,Roku Paid Subs,Billing Amount,Total Paid Subs,Revenue
0,2020,4,Apr,CONtv,26140,1208,4.99,27348,136466.52
1,2020,5,May,CONtv,27697,2279,4.99,29976,149580.24
2,2020,6,Jun,CONtv,32540,2613,4.99,35153,175413.47
3,2020,7,Jul,CONtv,34989,2431,4.99,37420,186725.80
4,2020,8,Aug,CONtv,35142,2232,4.99,37374,186496.26
5,2020,9,Sep,CONtv,34955,2009,4.99,36964,184450.36
6,2020,10,Oct,CONtv,35633,1844,4.99,37477,187010.23
7,2020,11,Nov,CONtv,36933,1833,4.99,38766,193442.34
8,2020,12,Dec,CONtv,36900,1947,4.99,38847,193846.53
9,2021,1,Jan,CONtv,37155,2229,4.99,39384,196526.16


# Amazon Content Data (Importing, Transforming)

In [23]:
CFilepath='C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/SVOD/Amazon/Amazon Content/'

Channels = ['Screambox', 'Docurama', 'CONtv', 'Dove Channel']
CountryF = ['US','UK']

In [24]:
#importing Content for all subs except Fandor

cont_lists = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022','2022-08':'2022','2022-09':'2022','2022-10':'2022','2022-11':'2022','2022-12':'2022',
         '2023-01':'2023','2023-02':'2023','2023-03':'2023'}
cl=[]

for i in Channels: 
    for key,value in cont_lists.items():
        cl = cl + [CFilepath + i +'/' + value + '/contusg_m_US_' + i + '_' + key + '.csv']
        

In [25]:
#importing Content for Fandor (Since UK data is always late)

cont_listsF = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022','2022-08':'2022','2022-09':'2022','2022-10':'2022','2022-11':'2022','2022-12':'2022',
         '2023-01':'2023','2023-02':'2023','2023-03':'2023'}
clF=[]

for i in CountryF: 
    for key,value in cont_listsF.items():
        clF = clF + [CFilepath + 'Fandor/' + i +'/' + value + '/contusg_m_' + i + '_' + 'Fandor' + '_' + key + '.csv']
        

In [26]:
dfcont = pd.concat([pd.read_csv(file) for file in cl]).reset_index()

In [27]:
dfFcont = pd.concat([pd.read_csv(file) for file in clF]).reset_index()
# Fandor_last= pd.read_csv(CFilepath + 'Fandor/US/2022/contusg_m_US_Fandor_2022-12.csv')
# dfFcont = pd.concat([dfFcont, Fandor_last])

In [28]:
dfFcont['Series or Movie Title']= dfFcont['Series or Movie Title'].fillna(dfFcont['Season or Movie Title'])

In [29]:
dfFcont= dfFcont.drop(columns=['Report Date','Season or Movie Title'])

In [30]:
Amz_cont=pd.concat([dfcont,dfFcont])

In [31]:
Amz_C = Amz_cont.loc[:,:]

In [32]:
Amz_C

,index,Report Start,Report End,Offer ID,Subscription,Vendor SKU,Content Type,Series or Movie Title,Season Number,Episode Number,Episode Name,Total Number of Streams,Total Minutes Streamed,Content Group,Total Customers Streamed
0,0,2020-04-01,2020-04-30,B073ZJYX6R-0M00000000,Screambox,6fe0d289_AOS,TV,Character with Christopher Nelson,1.0,1.0,"Halloween, Horror, and Jeff Goldblum's Balls",18,78.91,NaN,NaN
1,1,2020-04-01,2020-04-30,B073ZJYX6R-0M00000000,Screambox,6fe0d295_AOS,TV,Character with Christopher Nelson,1.0,2.0,Hollywood Politics and Dinnertime Prosthetics,5,41.18,NaN,NaN
2,2,2020-04-01,2020-04-30,B073ZJYX6R-0M00000000,Screambox,6fe0d299_AOS,TV,Character with Christopher Nelson,1.0,3.0,Where Are All the Women At?,5,46.30,NaN,NaN
3,3,2020-04-01,2020-04-30,B073ZJYX6R-0M00000000,Screambox,LAK730900,MOVIE,House,NaN,NaN,NaN,446,19874.90,NaN,NaN
4,4,2020-04-01,2020-04-30,B073ZJYX6R-0M00000000,Screambox,LAK731500,MOVIE,Return to Horror High,NaN,NaN,NaN,163,6359.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127613,21,2023-03-01,2023-03-31,B06XWCM4G6-1M00000000,Fandor,8626_the_pain_of_others,MOVIE,The Pain of Others,NaN,NaN,NaN,1,17.07,3P_SUBS,1.0
127614,22,2023-03-01,2023-03-31,B06XWCM4G6-1M00000000,Fandor,CEG10020677,MOVIE,Haunters: The Art Of The Scare,NaN,NaN,NaN,4,237.26,3P_SUBS,3.0
127615,23,2023-03-01,2023-03-31,B06XWCM4G6-1M00000000,Fandor,CEG10024051,MOVIE,Stay Human,NaN,NaN,NaN,3,53.31,3P_SUBS,2.0
127616,24,2023-03-01,2023-03-31,B06XWCM4G6-1M00000000,Fandor,NNVG10002283,MOVIE,Osterman Weekend,NaN,NaN,NaN,2,139.01,3P_SUBS,2.0


In [33]:
Amz_cont['Report Start']= pd.to_datetime(Amz_cont['Report Start'])
Amz_cont.loc[Amz_cont['Subscription']=='Dove Channel','Subscription']='Dove'
Amz_cont['Year']= Amz_cont['Report Start'].dt.year
Amz_cont['Month']= Amz_cont['Report Start'].dt.month
Amz_cont= Amz_cont.loc[:,('Year','Month','Subscription','Vendor SKU','Content Type','Series or Movie Title','Total Minutes Streamed','Total Number of Streams')]
Amz_cont.rename(columns={'Vendor SKU':'ID','Content Type':'TV/MOVIE','Series or Movie Title':'Title'}, inplace=True)

#There is multiple titles for the same Year/Month because its broken down by Payment type, as in paid subs vs free trial etc.

Amz_cont= Amz_cont.groupby(by=['Year','Month','Subscription','ID','TV/MOVIE','Title'])[['Total Minutes Streamed','Total Number of Streams']].agg(sum).reset_index()

In [34]:
Amz_cont['ID']= Amz_cont['ID'].astype(str) 
Amz_cont['Title']= Amz_cont['Title'].astype(str) 
Amz_cont['Subscription']= Amz_cont['Subscription'].astype(str) 
Amz_cont['TV/MOVIE']= Amz_cont['TV/MOVIE'].astype(str) 

In [35]:
ACo=Amz_cont.loc[Amz_cont['ID'].str[:3]=='008'].drop_duplicates()

In [36]:
ACo['ID2']= ACo['ID'].str[2:]

In [37]:
ACo

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams,ID2
1,2020,4,CONtv,00883476003771,TV,Super Mario Bros. Super Show,1650.25,149,883476003771
2,2020,4,CONtv,00883476003788,TV,Super Mario Bros. Super Show,1058.47,80,883476003788
3,2020,4,CONtv,00883476003795,TV,Super Mario Bros. Super Show,841.82,69,883476003795
4,2020,4,CONtv,00883476003801,TV,Super Mario Bros. Super Show,760.54,52,883476003801
5,2020,4,CONtv,00883476003818,TV,Super Mario Bros. Super Show,738.92,46,883476003818
...,...,...,...,...,...,...,...,...,...
266853,2023,2,Dove,00883476110417,MOVIE,Miracle Of Marcelino,3271.13,88,883476110417
272408,2023,3,CONtv,00883476007250,TV,Dino Squad,443.69,38,883476007250
272409,2023,3,CONtv,00883476025872,MOVIE,The Collector,3012.45,66,883476025872
277257,2023,3,Dove,00883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,11982.37,343,883476075594


In [38]:
Amz_cont=pd.merge(Amz_cont,ACo.loc[:,('ID','ID2')], on='ID', how='left')

In [39]:
Amz_cont['ID']=Amz_cont['ID2'].fillna(Amz_cont['ID'])
Amz_cont.drop_duplicates(inplace=True)

In [40]:
Amz_cont

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams,ID2
0,2020,4,CONtv,002HKR0M1FM,TV,Unsealed Alien Files,38.01,2,NaN
1,2020,4,CONtv,883476003771,TV,Super Mario Bros. Super Show,1650.25,149,883476003771
3,2020,4,CONtv,883476003788,TV,Super Mario Bros. Super Show,1058.47,80,883476003788
5,2020,4,CONtv,883476003795,TV,Super Mario Bros. Super Show,841.82,69,883476003795
7,2020,4,CONtv,883476003801,TV,Super Mario Bros. Super Show,760.54,52,883476003801
...,...,...,...,...,...,...,...,...,...
317570,2023,3,Screambox,TFD1000229,MOVIE,Elvira's Movie Macabre: They Came From Beyond ...,2484.38,71,NaN
317571,2023,3,Screambox,TFD1000230,MOVIE,Elvira's Movie Macabre: The Werewolf Of Washin...,1044.65,31,NaN
317572,2023,3,Screambox,TFD1000231,MOVIE,Elvira's Movie Macabre: The Doomsday Machine,3090.29,83,NaN
317573,2023,3,Screambox,TFD1000232,MOVIE,"Elvira's Movie Macabre: Gamera, Super Monster",637.68,45,NaN


In [41]:
#small fix

Amz_cont.loc[Amz_cont['Title']=='Sue Thomas: F.B.Eye','ID']='CEG10019302'

In [42]:
Amz_cont.loc[Amz_cont['Title'].str.contains('Sue Thomas',case=False)]

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams,ID2
4452,2020,4,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,8702.38,368,NaN
5397,2020,4,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,12732.01,520,NaN
5410,2020,4,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,25198.21,589,NaN
5411,2020,4,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,11427.05,425,NaN
5412,2020,4,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,10758.60,400,NaN
...,...,...,...,...,...,...,...,...,...
313195,2023,3,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,5787.38,206,NaN
313196,2023,3,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,5570.29,215,NaN
313197,2023,3,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,5454.73,203,NaN
313198,2023,3,Dove,CEG10019302,TV,Sue Thomas: F.B.Eye,5357.30,225,NaN


In [43]:
Amz_cont= Amz_cont.groupby(by=['Year','Month','Subscription','ID','TV/MOVIE','Title'])[['Total Minutes Streamed','Total Number of Streams']].agg(sum).reset_index()

In [44]:
Amz_cont= Amz_cont.loc[:,('Year','Month','Subscription','ID','TV/MOVIE','Title','Total Minutes Streamed','Total Number of Streams')]

In [45]:
Amz_cont

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams
0,2020,4,CONtv,002HKR0M1FM,TV,Unsealed Alien Files,38.01,2
1,2020,4,CONtv,038260,MOVIE,Dragon,41.62,4
2,2020,4,CONtv,069295,MOVIE,Naked Fear,7982.90,195
3,2020,4,CONtv,070291,MOVIE,"30,000 Leagues Under the Sea",879.23,32
4,2020,4,CONtv,100MillionBC,MOVIE,100 Million BC,1016.65,19
...,...,...,...,...,...,...,...,...
280934,2023,3,Screambox,TFD1000229,MOVIE,Elvira's Movie Macabre: They Came From Beyond ...,2484.38,71
280935,2023,3,Screambox,TFD1000230,MOVIE,Elvira's Movie Macabre: The Werewolf Of Washin...,1044.65,31
280936,2023,3,Screambox,TFD1000231,MOVIE,Elvira's Movie Macabre: The Doomsday Machine,3090.29,83
280937,2023,3,Screambox,TFD1000232,MOVIE,"Elvira's Movie Macabre: Gamera, Super Monster",637.68,45


In [46]:
Amz_cont.loc[Amz_cont['Title']=='11/11/11','Title']='11_11_11'

In [47]:
# Minutes Viewed_________________________

In [48]:
AmzMin = Amz_cont.loc[:,:].copy()

In [49]:
AmzMin = AmzMin.loc[AmzMin['Year']==2022,('Month','Subscription','Total Minutes Streamed')]

In [50]:
MonthMin = AmzMin.groupby(['Subscription','Month']).agg(sum).reset_index()

In [51]:
AvgMin = MonthMin.loc[:,('Subscription','Total Minutes Streamed')].groupby('Subscription').mean().reset_index()

In [52]:
AvgMin

,Subscription,Total Minutes Streamed
0,CONtv,2.282961e+06
1,Docurama,1.390716e+06
2,Dove,4.214540e+06
3,Fandor,2.204282e+06
4,Screambox,7.278226e+06


In [53]:
#Search
Amz_cont.loc[Amz_cont['Title'].str.contains('The Big Fix'),('Subscription','ID','TV/MOVIE','Title')].sort_values(by='ID').drop_duplicates()

,Subscription,ID,TV/MOVIE,Title
195804,Dove,1000000005964,MOVIE,The Big Fix


In [54]:
#sadfgsaf

# Roku Content Data (Importing, Transforming)

In [55]:
RContFilepath = 'C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/SVOD/Roku/Roku Content/'

In [56]:
R_Channels = ['CONtv','Docurama','Dove','Fandor','Screambox']

R_cont_lists = {'2020-04':'2020','2020-05':'2020','2020-06':'2020','2020-07':'2020','2020-08':'2020','2020-09':'2020','2020-10':'2020','2020-11':'2020','2020-12':'2020',
         '2021-01':'2021','2021-02':'2021','2021-03':'2021','2021-04':'2021','2021-05':'2021','2021-06':'2021','2021-07':'2021','2021-08':'2021','2021-09':'2021','2021-10':'2021','2021-11':'2021','2021-12':'2021',
         '2022-01':'2022','2022-02':'2022','2022-03':'2022','2022-04':'2022','2022-05':'2022','2022-06':'2022','2022-07':'2022','2022-08':'2022','2022-09':'2022','2022-10':'2022','2022-11':'2022','2022-12':'2022',
         '2023-01':'2023','2023-02':'2023','2023-03':'2023'}

In [57]:
R_HOV = []
R_STR = []

for i in R_Channels:
    for key,value in R_cont_lists.items():
        R_HOV = R_HOV + [RContFilepath + i + '/HOV/' + value + '/Roku SVOD ' + i + ' HOV ' + key + '.xlsx']
        R_STR = R_STR + [RContFilepath + i + '/Streams/' + value + '/Roku SVOD ' + i + ' Streams ' + key + '.xlsx']        

In [58]:
Rok_Hrs = pd.concat([pd.read_excel(file) for file in R_HOV])

In [59]:
Rok_Str = pd.concat([pd.read_excel(file) for file in R_STR])

In [60]:
glossaryAR = pd.read_excel('C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/SVOD/Roku/Mapping Table 2023_03.xlsx')

In [61]:
Rok_Hrs

,Channel,TV/MOVIE,Title,Content Title,Season Number,Episode Number,HOV,Date
0,CONtv,MOVIE,11_11_11,11_11_11,NaN,NaN,7.05000,2020-04-01
1,CONtv,MOVIE,100 Degrees Below Zero,100 Degrees Below Zero,NaN,NaN,16.93333,2020-04-01
2,CONtv,MOVIE,100 Ghost Street: The Return of Richard Speck,100 Ghost Street: The Return of Richard Speck,NaN,NaN,14.09166,2020-04-01
3,CONtv,MOVIE,100 Million BC,100 Million BC,NaN,NaN,18.60555,2020-04-01
4,CONtv,MOVIE,13/13/13,13/13/13,NaN,NaN,8.91944,2020-04-01
...,...,...,...,...,...,...,...,...
1015,Screambox,MOVIE,Final Move,Final Move,NaN,NaN,0.00277,2023-03-01
1016,Screambox,MOVIE,Mask of Thorn,Mask of Thorn,NaN,NaN,0.00277,2023-03-01
1017,Screambox,MOVIE,Masters of Horror: Incident On and Off a Mount...,Masters of Horror: Incident On and Off a Mount...,NaN,NaN,0.00277,2023-03-01
1018,Screambox,MOVIE,Mind Rage,Mind Rage,NaN,NaN,0.00277,2023-03-01


In [62]:
Rok_Str

,Channel,TV/MOVIE,Title,Content Title,Season Number,Episode Number,Streams,Date
0,CONtv,MOVIE,11_11_11,11_11_11,NaN,NaN,9.0,2020-04-01
1,CONtv,MOVIE,100 Degrees Below Zero,100 Degrees Below Zero,NaN,NaN,18.0,2020-04-01
2,CONtv,MOVIE,100 Ghost Street: The Return of Richard Speck,100 Ghost Street: The Return of Richard Speck,NaN,NaN,15.0,2020-04-01
3,CONtv,MOVIE,100 Million BC,100 Million BC,NaN,NaN,26.0,2020-04-01
4,CONtv,MOVIE,13/13/13,13/13/13,NaN,NaN,16.0,2020-04-01
...,...,...,...,...,...,...,...,...
1015,Screambox,MOVIE,The Phantom of the Opera,The Phantom of the Opera,NaN,NaN,1.0,2023-03-01
1016,Screambox,MOVIE,Timber Falls,Timber Falls,NaN,NaN,1.0,2023-03-01
1017,Screambox,MOVIE,Tokoloshe: An African Curse,Tokoloshe: An African Curse,NaN,NaN,1.0,2023-03-01
1018,Screambox,MOVIE,Torture Ship,Torture Ship,NaN,NaN,1.0,2023-03-01


In [63]:
glossaryAR

,Subscription,TV/MOVIE,Amazon ID,Amazon Title,Roku Title,A or R Title,Common ID,Check
0,Fandor,MOVIE,1000000005816,1:54,1:54,1:54,1000000005816,Check
1,Fandor,MOVIE,4960_8,8,NaN,8,4960_8,Check
2,Screambox,MOVIE,NaN,NaN,8,8,4960_8,Check
3,Fandor,MOVIE,5379_74_78,74.78,NaN,74.78,5379_74_78,Check
4,CONtv,MOVIE,CEG10018116,11/11/11,11/11/11,11/11/11,CEG10018116,Check
...,...,...,...,...,...,...,...,...
20539,Screambox,MOVIE,NaN,NaN,Evil Little Things,Evil Little Things,1000000008280,Check
20540,Screambox,MOVIE,NaN,NaN,Family Dinner,Family Dinner,1000000009189,Check
20541,Screambox,MOVIE,NaN,NaN,FeardotCom,FeardotCom,1000000008268,Check
20542,Screambox,MOVIE,NaN,NaN,Razorback,Razorback,1000000011805,Check


In [64]:
#Hours
Rok_Hrs['Date']=pd.to_datetime(Rok_Hrs['Date'])
Rok_Hrs['Year']=Rok_Hrs['Date'].dt.year
Rok_Hrs['Month']=Rok_Hrs['Date'].dt.month
Rok_Hrs['Total Minutes Viewed']= Rok_Hrs['HOV']*60
Rok_Hrs.drop(columns=['HOV'], inplace=True)
Rok_Hrs.rename(columns={'Channel':'Subscription','Total Minutes Viewed':'Roku Tot Min Viewed'},inplace=True)
Rok_Hrs= Rok_Hrs.groupby(by=['Date','Year','Month','Subscription','TV/MOVIE','Title'])[['Roku Tot Min Viewed']].agg(sum).reset_index()

#Streams
Rok_Str['Date']=pd.to_datetime(Rok_Str['Date'])
Rok_Str['Year']=Rok_Str['Date'].dt.year
Rok_Str['Month']=Rok_Str['Date'].dt.month
Rok_Str.rename(columns={'Channel':'Subscription','Streams':'Roku Streams'},inplace=True)
Rok_Str= Rok_Str.groupby(by=['Date','Year','Month','Subscription','TV/MOVIE','Title'])[['Roku Streams']].agg(sum).reset_index()

In [65]:
#Merge Minutes And Streams
ContR= pd.merge(Rok_Hrs, Rok_Str, on=('Date','Year','Month','Subscription','TV/MOVIE','Title'), how='left')

In [66]:
# Minutes Viewed_________________________

In [67]:
RokMin = ContR.loc[:,:].copy()

In [68]:
RokMin = RokMin.loc[RokMin['Year']==2022,('Month','Subscription','Roku Tot Min Viewed')]

In [69]:
RMonthMin = RokMin.groupby(['Subscription','Month']).agg(sum).reset_index()

In [70]:
RAvgMin = RMonthMin.loc[:,('Subscription','Roku Tot Min Viewed')].groupby('Subscription').mean().reset_index()

In [71]:
RAvgMin

,Subscription,Roku Tot Min Viewed
0,CONtv,2.483718e+06
1,Docurama,6.306599e+05
2,Dove,4.158054e+06
3,Fandor,5.912148e+05
4,Screambox,1.739429e+06


In [72]:
ContR

,Date,Year,Month,Subscription,TV/MOVIE,Title,Roku Tot Min Viewed,Roku Streams
0,2020-04-01,2020,4,CONtv,MOVIE,100 Degrees Below Zero,1015.9998,18.0
1,2020-04-01,2020,4,CONtv,MOVIE,100 Ghost Street: The Return of Richard Speck,845.4996,15.0
2,2020-04-01,2020,4,CONtv,MOVIE,100 Million BC,1116.3330,26.0
3,2020-04-01,2020,4,CONtv,MOVIE,11_11_11,423.0000,9.0
4,2020-04-01,2020,4,CONtv,MOVIE,13/13/13,535.1664,16.0
...,...,...,...,...,...,...,...,...
81880,2023-03-01,2023,3,Screambox,TV,The Dead Zone,8697.8082,242.0
81881,2023-03-01,2023,3,Screambox,TV,The Island,10025.9964,333.0
81882,2023-03-01,2023,3,Screambox,TV,Toxic Crusaders,893.4966,63.0
81883,2023-03-01,2023,3,Screambox,TV,Welcome to Daisyland,432.9990,90.0


In [73]:
ContR.fillna(0, inplace=True)

# Merge Both AMAZON and ROKU

In [74]:
glossaryAR

,Subscription,TV/MOVIE,Amazon ID,Amazon Title,Roku Title,A or R Title,Common ID,Check
0,Fandor,MOVIE,1000000005816,1:54,1:54,1:54,1000000005816,Check
1,Fandor,MOVIE,4960_8,8,NaN,8,4960_8,Check
2,Screambox,MOVIE,NaN,NaN,8,8,4960_8,Check
3,Fandor,MOVIE,5379_74_78,74.78,NaN,74.78,5379_74_78,Check
4,CONtv,MOVIE,CEG10018116,11/11/11,11/11/11,11/11/11,CEG10018116,Check
...,...,...,...,...,...,...,...,...
20539,Screambox,MOVIE,NaN,NaN,Evil Little Things,Evil Little Things,1000000008280,Check
20540,Screambox,MOVIE,NaN,NaN,Family Dinner,Family Dinner,1000000009189,Check
20541,Screambox,MOVIE,NaN,NaN,FeardotCom,FeardotCom,1000000008268,Check
20542,Screambox,MOVIE,NaN,NaN,Razorback,Razorback,1000000011805,Check


In [75]:
Amz_cont

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams
0,2020,4,CONtv,002HKR0M1FM,TV,Unsealed Alien Files,38.01,2
1,2020,4,CONtv,038260,MOVIE,Dragon,41.62,4
2,2020,4,CONtv,069295,MOVIE,Naked Fear,7982.90,195
3,2020,4,CONtv,070291,MOVIE,"30,000 Leagues Under the Sea",879.23,32
4,2020,4,CONtv,100MillionBC,MOVIE,100 Million BC,1016.65,19
...,...,...,...,...,...,...,...,...
280934,2023,3,Screambox,TFD1000229,MOVIE,Elvira's Movie Macabre: They Came From Beyond ...,2484.38,71
280935,2023,3,Screambox,TFD1000230,MOVIE,Elvira's Movie Macabre: The Werewolf Of Washin...,1044.65,31
280936,2023,3,Screambox,TFD1000231,MOVIE,Elvira's Movie Macabre: The Doomsday Machine,3090.29,83
280937,2023,3,Screambox,TFD1000232,MOVIE,"Elvira's Movie Macabre: Gamera, Super Monster",637.68,45


In [76]:
ContR.loc[ContR['Title'].str.contains('Masters of Horror'),'TV/MOVIE']='TV'

In [77]:
ContR = pd.merge(ContR, glossaryAR.loc[:,('Subscription','TV/MOVIE','Roku Title','Common ID')], left_on=('Subscription','TV/MOVIE','Title'), right_on=('Subscription','TV/MOVIE','Roku Title'), how='left')

In [78]:
glossaryAR['Amazon ID']=glossaryAR['Amazon ID'].astype(str)
Amz_cont['ID']=Amz_cont['ID'].astype(str)

Amz_cont = pd.merge(Amz_cont, glossaryAR.loc[:,('Subscription','TV/MOVIE','Roku Title','Amazon ID','Common ID')], left_on=('Subscription','TV/MOVIE','ID'), right_on=('Subscription','TV/MOVIE','Amazon ID'), how='left')
Amz_cont = Amz_cont.loc[:,('Year','Month','Subscription','Common ID','TV/MOVIE','Title','Total Minutes Streamed','Total Number of Streams')].drop_duplicates()
Amz_cont.rename(columns={'Common ID':'ID'},inplace=True)

In [79]:
Amz_cont

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams
0,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,38.01,2
1,2020,4,CONtv,CEG10018197,MOVIE,Dragon,41.62,4
2,2020,4,CONtv,CEG10023839,MOVIE,Naked Fear,7982.90,195
3,2020,4,CONtv,CEG10018124,MOVIE,"30,000 Leagues Under the Sea",879.23,32
4,2020,4,CONtv,CEG10018118,MOVIE,100 Million BC,1016.65,19
...,...,...,...,...,...,...,...,...
304487,2023,3,Screambox,TFD1000229,MOVIE,Elvira's Movie Macabre: They Came From Beyond ...,2484.38,71
304488,2023,3,Screambox,TFD1000230,MOVIE,Elvira's Movie Macabre: The Werewolf Of Washin...,1044.65,31
304489,2023,3,Screambox,TFD1000231,MOVIE,Elvira's Movie Macabre: The Doomsday Machine,3090.29,83
304490,2023,3,Screambox,TFD1000232,MOVIE,"Elvira's Movie Macabre: Gamera, Super Monster",637.68,45


In [80]:
ContR.rename(columns={'Common ID':'ID'},inplace=True)

In [81]:
ContR['ID']=ContR['ID'].astype(str)
Amz_cont['ID']=Amz_cont['ID'].astype(str)

In [82]:
ContR

,Date,Year,Month,Subscription,TV/MOVIE,Title,Roku Tot Min Viewed,Roku Streams,Roku Title,ID
0,2020-04-01,2020,4,CONtv,MOVIE,100 Degrees Below Zero,1015.9998,18.0,100 Degrees Below Zero,CEG10018117
1,2020-04-01,2020,4,CONtv,MOVIE,100 Ghost Street: The Return of Richard Speck,845.4996,15.0,100 Ghost Street: The Return of Richard Speck,CEG10018177
2,2020-04-01,2020,4,CONtv,MOVIE,100 Million BC,1116.3330,26.0,100 Million BC,CEG10018118
3,2020-04-01,2020,4,CONtv,MOVIE,11_11_11,423.0000,9.0,11_11_11,CEG10018116
4,2020-04-01,2020,4,CONtv,MOVIE,11_11_11,423.0000,9.0,11_11_11,CEG10018116
...,...,...,...,...,...,...,...,...,...,...
85620,2023-03-01,2023,3,Screambox,TV,The Dead Zone,8697.8082,242.0,The Dead Zone,THEDEADZONE
85621,2023-03-01,2023,3,Screambox,TV,The Island,10025.9964,333.0,The Island,THEISLAND
85622,2023-03-01,2023,3,Screambox,TV,Toxic Crusaders,893.4966,63.0,Toxic Crusaders,TOXICCRUSADERS
85623,2023-03-01,2023,3,Screambox,TV,Welcome to Daisyland,432.9990,90.0,Welcome to Daisyland,WELCOMETODAISYLAND


In [83]:
Amz_cont

,Year,Month,Subscription,ID,TV/MOVIE,Title,Total Minutes Streamed,Total Number of Streams
0,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,38.01,2
1,2020,4,CONtv,CEG10018197,MOVIE,Dragon,41.62,4
2,2020,4,CONtv,CEG10023839,MOVIE,Naked Fear,7982.90,195
3,2020,4,CONtv,CEG10018124,MOVIE,"30,000 Leagues Under the Sea",879.23,32
4,2020,4,CONtv,CEG10018118,MOVIE,100 Million BC,1016.65,19
...,...,...,...,...,...,...,...,...
304487,2023,3,Screambox,TFD1000229,MOVIE,Elvira's Movie Macabre: They Came From Beyond ...,2484.38,71
304488,2023,3,Screambox,TFD1000230,MOVIE,Elvira's Movie Macabre: The Werewolf Of Washin...,1044.65,31
304489,2023,3,Screambox,TFD1000231,MOVIE,Elvira's Movie Macabre: The Doomsday Machine,3090.29,83
304490,2023,3,Screambox,TFD1000232,MOVIE,"Elvira's Movie Macabre: Gamera, Super Monster",637.68,45


In [84]:
Content = pd.merge(Amz_cont,ContR.loc[:,('Year','Month','Subscription','TV/MOVIE','ID','Title','Roku Tot Min Viewed','Roku Streams')], on=['Subscription','Year','Month','TV/MOVIE','ID'], how='outer').drop_duplicates()

In [85]:
Content.head()

,Year,Month,Subscription,ID,TV/MOVIE,Title_x,Total Minutes Streamed,Total Number of Streams,Title_y,Roku Tot Min Viewed,Roku Streams
0,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,38.01,2.0,Unsealed Alien Files - Season 1,190.4976,15.0
2,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,19.39,1.0,Unsealed Alien Files - Season 1,190.4976,15.0
4,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,37.79,2.0,Unsealed Alien Files - Season 1,190.4976,15.0
6,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,20.81,2.0,Unsealed Alien Files - Season 1,190.4976,15.0
8,2020,4,CONtv,UNSEALEDALIENFILES,TV,Unsealed Alien Files,37.89,2.0,Unsealed Alien Files - Season 1,190.4976,15.0


In [86]:
Content.rename(columns={'Total Minutes Streamed':'Amzn Min Streamed','Total Number of Streams':'Amzn Streams','Roku Tot Min Viewed':'Roku Min Streamed'}, inplace = True)
Content['Title']=Content['Title_x'].fillna(Content['Title_y'])

Content['Amzn Min Streamed']= Content['Amzn Min Streamed'].fillna(0)
Content['Amzn Streams']= Content['Amzn Streams'].fillna(0)
Content['Roku Min Streamed']= Content['Roku Min Streamed'].fillna(0)
Content['Roku Streams']= Content['Roku Streams'].fillna(0)


Content['Total Min Streamed']= Content['Amzn Min Streamed']+ Content['Roku Min Streamed']
Content['Total Streams']= Content['Amzn Streams']+ Content['Roku Streams']

Content= Content.loc[:,('Year','Month','Subscription','ID','TV/MOVIE','Title','Amzn Min Streamed','Amzn Streams','Roku Min Streamed','Roku Streams','Total Min Streamed','Total Streams')]
Content=Content.sort_values(by=['Subscription','Title']).reset_index(drop=True)

In [87]:
Content

,Year,Month,Subscription,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Roku Min Streamed,Roku Streams,Total Min Streamed,Total Streams
0,2020,4,CONtv,CEG10018117,MOVIE,100 Below Zero,89.02,2.0,1015.9998,18.0,1105.0198,20.0
1,2020,5,CONtv,CEG10018117,MOVIE,100 Below Zero,3.38,1.0,717.3330,17.0,720.7130,18.0
2,2020,6,CONtv,CEG10018117,MOVIE,100 Below Zero,93.13,6.0,442.9998,21.0,536.1298,27.0
3,2020,7,CONtv,CEG10018117,MOVIE,100 Below Zero,97.88,3.0,849.3330,23.0,947.2130,26.0
4,2020,8,CONtv,CEG10018117,MOVIE,100 Below Zero,118.56,6.0,387.3330,8.0,505.8930,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
323739,2021,5,Screambox,SBOX100000109,MOVIE,e-Demon,0.00,0.0,798.1662,19.0,798.1662,19.0
323740,2021,6,Screambox,SBOX100000109,MOVIE,e-Demon,0.00,0.0,861.6666,16.0,861.6666,16.0
323741,2021,7,Screambox,SBOX100000109,MOVIE,e-Demon,0.00,0.0,521.6664,16.0,521.6664,16.0
323742,2021,8,Screambox,SBOX100000109,MOVIE,e-Demon,0.00,0.0,631.5000,24.0,631.5000,24.0


In [88]:
#Content['TITLE']=Content.groupby('ID')['Title'].transform('first')
Content['ID']= Content['ID'].astype(str) 
Content['Title']= Content['Title'].astype(str) 
Content['Subscription']= Content['Subscription'].astype(str) 
Content['TV/MOVIE']= Content['TV/MOVIE'].astype(str) 

In [89]:
#Content['Title']= Content.groupby('ID')['Title'].transform('first')

In [90]:
#Search
Content.loc[Content['Title'].str.contains('The Big Fix'),:]

,Year,Month,Subscription,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Roku Min Streamed,Roku Streams,Total Min Streamed,Total Streams
196895,2022,7,Dove,1000000005964,MOVIE,The Big Fix,7995.24,161.0,0.0000,0.0,7995.2400,161.0
196896,2022,8,Dove,1000000005964,MOVIE,The Big Fix,4933.93,95.0,0.0000,0.0,4933.9300,95.0
196897,2022,9,Dove,1000000005964,MOVIE,The Big Fix,3043.91,58.0,0.0000,0.0,3043.9100,58.0
196898,2022,10,Dove,1000000005964,MOVIE,The Big Fix,2504.76,58.0,0.0000,0.0,2504.7600,58.0
196899,2022,11,Dove,1000000005964,MOVIE,The Big Fix,2488.26,48.0,0.0000,0.0,2488.2600,48.0
196900,2022,12,Dove,1000000005964,MOVIE,The Big Fix,3131.18,79.0,2.6664,1.0,3133.8464,80.0
196901,2023,1,Dove,1000000005964,MOVIE,The Big Fix,5269.75,97.0,0.0000,0.0,5269.7500,97.0
196902,2023,2,Dove,1000000005964,MOVIE,The Big Fix,3096.19,70.0,0.0000,0.0,3096.1900,70.0
196903,2023,3,Dove,1000000005964,MOVIE,The Big Fix,1963.71,42.0,0.0000,0.0,1963.7100,42.0


# Content without ID, Groups the episodes together

In [91]:
#Small Fix

Content.loc[Content['ID']=='1000000002887','Title']='Masters of Horror'

In [92]:
Content2=Content.loc[:,:]
Content2=Content2.groupby(by=['Subscription','Year','Month','TV/MOVIE','ID']).agg(sum).reset_index()

In [93]:
Idt=Content.loc[:,('Subscription','ID','Title','TV/MOVIE')]

In [94]:
Idt.drop_duplicates(inplace=True)

In [95]:
Idt.drop_duplicates(subset=('Subscription','TV/MOVIE','ID'), keep='first', inplace=True)

In [96]:
Idt

,Subscription,ID,Title,TV/MOVIE
0,CONtv,CEG10018117,100 Below Zero,MOVIE
36,CONtv,CEG10018177,100 Ghost Street: The Return of Richard Speck,MOVIE
72,CONtv,CEG10018118,100 Million BC,MOVIE
108,CONtv,1000000003336,11 Blocks,MOVIE
116,CONtv,CEG10018116,11/11/11,MOVIE
...,...,...,...,...
323671,Screambox,CEG10028242,Zombies,MOVIE
323686,Screambox,SBOX100000060,_DUPE_An English Haunting,MOVIE
323692,Screambox,SBOX100000095,_DUPE_Covenant,MOVIE
323696,Screambox,SBOX100000167,_DUPE_Marla,MOVIE


In [97]:
Content2=pd.merge(Content2,Idt, on=('Subscription','TV/MOVIE','ID'), how='left')

In [98]:
Content2=Content2.groupby(by=['Year','Month','Subscription','TV/MOVIE','Title']).agg(sum).reset_index()

Idt2=Content.loc[:,('Subscription','ID','Title','TV/MOVIE')]
Idt2.drop_duplicates(inplace=True)
Idt2.drop_duplicates(subset=('Subscription','TV/MOVIE','Title'), keep='last', inplace=True)

Content2=pd.merge(Content2,Idt2, on=('Subscription','TV/MOVIE','Title'), how='left')

In [99]:
Idt2

,Subscription,ID,Title,TV/MOVIE
0,CONtv,CEG10018117,100 Below Zero,MOVIE
36,CONtv,CEG10018177,100 Ghost Street: The Return of Richard Speck,MOVIE
72,CONtv,CEG10018118,100 Million BC,MOVIE
108,CONtv,1000000003336,11 Blocks,MOVIE
116,CONtv,CEG10018116,11/11/11,MOVIE
...,...,...,...,...
323686,Screambox,SBOX100000060,_DUPE_An English Haunting,MOVIE
323692,Screambox,SBOX100000095,_DUPE_Covenant,MOVIE
323696,Screambox,SBOX100000167,_DUPE_Marla,MOVIE
323702,Screambox,CEG10018962,_DUPE_The Perfect Husband,MOVIE


In [100]:
Content2

,Year,Month,Subscription,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Roku Min Streamed,Roku Streams,Total Min Streamed,Total Streams,ID
0,2020,4,CONtv,MOVIE,100 Below Zero,89.02,2.0,1015.9998,18.0,1105.0198,20.0,CEG10018117
1,2020,4,CONtv,MOVIE,100 Ghost Street: The Return of Richard Speck,1.08,1.0,845.4996,15.0,846.5796,16.0,CEG10018177
2,2020,4,CONtv,MOVIE,100 Million BC,1016.65,19.0,1116.3330,26.0,2132.9830,45.0,CEG10018118
3,2020,4,CONtv,MOVIE,11/11/11,292.05,7.0,423.0000,9.0,715.0500,16.0,CEG10018116
4,2020,4,CONtv,MOVIE,13 Demons,716.62,23.0,0.0000,0.0,716.6200,23.0,CEG10020241
...,...,...,...,...,...,...,...,...,...,...,...,...
165710,2023,3,Screambox,TV,The Dead Zone,10868.09,394.0,452286.0264,12584.0,463154.1164,12978.0,THEDEADZONE
165711,2023,3,Screambox,TV,The Island,45435.87,1651.0,140363.9496,4662.0,185799.8196,6313.0,THEISLAND
165712,2023,3,Screambox,TV,Toxic Crusaders,7324.44,480.0,11615.4558,819.0,18939.8958,1299.0,TOXICCRUSADERS
165713,2023,3,Screambox,TV,Welcome to Daisyland,0.00,0.0,432.9990,90.0,432.9990,90.0,WELCOMETODAISYLAND


In [101]:
Cond1=[
    Content2['Month']==1, Content2['Month']==2, Content2['Month']==3, Content2['Month']==4,
    Content2['Month']==5, Content2['Month']==6, Content2['Month']==7, Content2['Month']==8,
    Content2['Month']==9, Content2['Month']==10, Content2['Month']==11, Content2['Month']==12
]
Months_names1 = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Content2['Month Abv'] = np.select(Cond1,Months_names1, default=np.nan)

In [102]:
Content2['Amzn Min/Stream']= Content2['Amzn Min Streamed']/ Content2['Amzn Streams']
Content2['Roku Min/Stream']= Content2['Roku Min Streamed']/ Content2['Roku Streams']
Content2['Min/Stream']= Content2['Total Min Streamed']/ Content2['Total Streams']

# Rev, Percentile, Total Channel Minutes

In [103]:
#Total Minutes per Month
TotM=Content2.groupby(by=['Subscription','Year','Month'])[['Total Min Streamed']].agg(sum).reset_index()

#Total Rev
RevM=Subs.loc[:,('Year','Month(num)','Subscription','Revenue')]

#Merge
TotMRevM=pd.merge(TotM,RevM, left_on=('Subscription','Year','Month'), right_on=('Subscription','Year','Month(num)'), how='left')

In [104]:
TotMRevM.rename(columns={'Total Min Streamed':'Tot Sub Month Min','Revenue':'Tot Sub Month Rev'}, inplace=True)
TotMRevM= TotMRevM.loc[:,('Subscription','Year','Month','Tot Sub Month Min','Tot Sub Month Rev')]

In [105]:
Subs.head()

,Year,Month(num),Month,Subscription,Amazon Paid Subs,Roku Paid Subs,Billing Amount,Total Paid Subs,Revenue
0,2020,4,Apr,CONtv,26140,1208,4.99,27348,136466.52
1,2020,5,May,CONtv,27697,2279,4.99,29976,149580.24
2,2020,6,Jun,CONtv,32540,2613,4.99,35153,175413.47
3,2020,7,Jul,CONtv,34989,2431,4.99,37420,186725.80
4,2020,8,Aug,CONtv,35142,2232,4.99,37374,186496.26


In [106]:
RevM.head()

,Year,Month(num),Subscription,Revenue
0,2020,4,CONtv,136466.52
1,2020,5,CONtv,149580.24
2,2020,6,CONtv,175413.47
3,2020,7,CONtv,186725.80
4,2020,8,CONtv,186496.26


In [107]:
TotMRevM.head()

,Subscription,Year,Month,Tot Sub Month Min,Tot Sub Month Rev
0,CONtv,2020,4,1.993045e+07,136466.52
1,CONtv,2020,5,2.214802e+07,149580.24
2,CONtv,2020,6,1.610129e+07,175413.47
3,CONtv,2020,7,1.498874e+07,186725.80
4,CONtv,2020,8,1.290963e+07,186496.26


In [108]:
Content2= pd.merge(Content2, TotMRevM, on=('Subscription','Year','Month'), how='left')

In [109]:
Content2['PerTM']=Content2['Total Min Streamed']/Content2['Tot Sub Month Min']
Content2['Content Month Rev']=Content2['PerTM']*Content2['Tot Sub Month Rev']
Content2['Percentile Rank (Min)']=Content2.groupby(by=['Subscription','Year','Month'])['Total Min Streamed'].rank(pct=True)

In [110]:
Content2= Content2.loc[:,('Subscription','Year','Month','Month Abv','ID','TV/MOVIE','Title',
                          'Amzn Min Streamed','Amzn Streams','Amzn Min/Stream','Roku Min Streamed','Roku Streams','Roku Min/Stream',
                          'Total Min Streamed','Percentile Rank (Min)','Total Streams','Min/Stream','Content Month Rev')]

In [111]:
Content2.head(20)

,Subscription,Year,Month,Month Abv,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Roku Min Streamed,Roku Streams,Roku Min/Stream,Total Min Streamed,Percentile Rank (Min),Total Streams,Min/Stream,Content Month Rev
0,CONtv,2020,4,Apr,CEG10018117,MOVIE,100 Below Zero,89.02,2.0,44.510000,1015.9998,18.0,56.444433,1105.0198,0.614765,20.0,55.250990,7.566223
1,CONtv,2020,4,Apr,CEG10018177,MOVIE,100 Ghost Street: The Return of Richard Speck,1.08,1.0,1.080000,845.4996,15.0,56.366640,846.5796,0.577181,16.0,52.911225,5.796647
2,CONtv,2020,4,Apr,CEG10018118,MOVIE,100 Million BC,1016.65,19.0,53.507895,1116.3330,26.0,42.935885,2132.9830,0.718121,45.0,47.399622,14.604829
3,CONtv,2020,4,Apr,CEG10018116,MOVIE,11/11/11,292.05,7.0,41.721429,423.0000,9.0,47.000000,715.0500,0.551678,16.0,44.690625,4.896046
4,CONtv,2020,4,Apr,CEG10020241,MOVIE,13 Demons,716.62,23.0,31.157391,0.0000,0.0,NaN,716.6200,0.553020,23.0,31.157391,4.906796
5,CONtv,2020,4,Apr,CEG10018119,MOVIE,13/13/13,80.40,3.0,26.800000,535.1664,16.0,33.447900,615.5664,0.538255,19.0,32.398232,4.214868
6,CONtv,2020,4,Apr,CEG10027188,MOVIE,20 Feet Below,15075.16,388.0,38.853505,313.3332,8.0,39.166650,15388.4932,0.934228,396.0,38.859831,105.367138
7,CONtv,2020,4,Apr,CEG10018121,MOVIE,2008 - Doomsday,473.92,13.0,36.455385,382.9998,9.0,42.555533,856.9198,0.578523,22.0,38.950900,5.867448
8,CONtv,2020,4,Apr,CEG10018178,MOVIE,2010 Moby Dick,167.36,3.0,55.786667,5648.1666,130.0,43.447435,5815.5266,0.844295,133.0,43.725764,39.819714
9,CONtv,2020,4,Apr,CEG10018122,MOVIE,2012: Ice Age,654.40,10.0,65.440000,795.3330,17.0,46.784294,1449.7330,0.656376,27.0,53.693815,9.926522


# Trend Month by Month

In [112]:
Content3= Content2.loc[:,:]

In [113]:
Content3['Month'] = Content3['Month'].astype(str).str.zfill(2)
#Content3['Month'] = Content3['Month'].zfill(2)

In [114]:
#Content3['Date']= Content3['Month Abv']+'-'+Content3['Year'].astype(str)
Content3['Date']= Content3['Year'].astype(str)+'-'+Content3['Month'].astype(str)

In [115]:
Content3

,Subscription,Year,Month,Month Abv,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Roku Min Streamed,Roku Streams,Roku Min/Stream,Total Min Streamed,Percentile Rank (Min),Total Streams,Min/Stream,Content Month Rev,Date
0,CONtv,2020,04,Apr,CEG10018117,MOVIE,100 Below Zero,89.02,2.0,44.510000,1015.9998,18.0,56.444433,1105.0198,0.614765,20.0,55.250990,7.566223,2020-04
1,CONtv,2020,04,Apr,CEG10018177,MOVIE,100 Ghost Street: The Return of Richard Speck,1.08,1.0,1.080000,845.4996,15.0,56.366640,846.5796,0.577181,16.0,52.911225,5.796647,2020-04
2,CONtv,2020,04,Apr,CEG10018118,MOVIE,100 Million BC,1016.65,19.0,53.507895,1116.3330,26.0,42.935885,2132.9830,0.718121,45.0,47.399622,14.604829,2020-04
3,CONtv,2020,04,Apr,CEG10018116,MOVIE,11/11/11,292.05,7.0,41.721429,423.0000,9.0,47.000000,715.0500,0.551678,16.0,44.690625,4.896046,2020-04
4,CONtv,2020,04,Apr,CEG10020241,MOVIE,13 Demons,716.62,23.0,31.157391,0.0000,0.0,NaN,716.6200,0.553020,23.0,31.157391,4.906796,2020-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165710,Screambox,2023,03,Mar,THEDEADZONE,TV,The Dead Zone,10868.09,394.0,27.583985,452286.0264,12584.0,35.941356,463154.1164,0.995402,12978.0,35.687634,16350.410936,2023-03
165711,Screambox,2023,03,Mar,THEISLAND,TV,The Island,45435.87,1651.0,27.520212,140363.9496,4662.0,30.108097,185799.8196,0.989655,6313.0,29.431304,6559.163127,2023-03
165712,Screambox,2023,03,Mar,TOXICCRUSADERS,TV,Toxic Crusaders,7324.44,480.0,15.259250,11615.4558,819.0,14.182486,18939.8958,0.862069,1299.0,14.580366,668.622103,2023-03
165713,Screambox,2023,03,Mar,WELCOMETODAISYLAND,TV,Welcome to Daisyland,0.00,0.0,NaN,432.9990,90.0,4.811100,432.9990,0.254023,90.0,4.811100,15.285866,2023-03


In [116]:
CMSS= Content3.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Total Min Streamed']).reset_index()
CMSSM= Content3.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Min/Stream']).reset_index()
RPP= Content3.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Percentile Rank (Min)']).reset_index()

In [117]:
CMSS['Metric']='Minutes Streamed'
CMSSM['Metric']='Min/Stream'
RPP['Metric']='Percentile Rank (Min)'

In [118]:
ColumnD = list(set(Content3['Date'].unique()))

In [119]:
ColumnD.sort()

In [120]:
ColumnD

['2020-04',
 '2020-05',
 '2020-06',
 '2020-07',
 '2020-08',
 '2020-09',
 '2020-10',
 '2020-11',
 '2020-12',
 '2021-01',
 '2021-02',
 '2021-03',
 '2021-04',
 '2021-05',
 '2021-06',
 '2021-07',
 '2021-08',
 '2021-09',
 '2021-10',
 '2021-11',
 '2021-12',
 '2022-01',
 '2022-02',
 '2022-03',
 '2022-04',
 '2022-05',
 '2022-06',
 '2022-07',
 '2022-08',
 '2022-09',
 '2022-10',
 '2022-11',
 '2022-12',
 '2023-01',
 '2023-02',
 '2023-03']

In [121]:
CMSS

Subscription                   ID TV/MOVIE  \
Date                                              
0           CONtv        1000000000019    MOVIE   
1           CONtv        1000000000029    MOVIE   
2           CONtv        1000000000180    MOVIE   
3           CONtv        1000000000196    MOVIE   
4           CONtv        1000000000197    MOVIE   
...           ...                  ...      ...   
9573    Screambox   WELCOMETODAISYLAND       TV   
9574    Screambox        WEREWOLFFEVER    MOVIE   
9575    Screambox          WITHOUTNAME    MOVIE   
9576    Screambox  YOURESOCOOLBREWSTER       TV   
9577    Screambox          ZOMBIHORROR    MOVIE   

                                                  Title Total Min Streamed  \
Date                                                               2020-04   
0                                        The Astrologer                NaN   
1                                       Psychos In Love                NaN   
2                             Guardian Of The Highlands                NaN   
3                                              The Baby                NaN   
4                                       Bad Man's River                NaN   
...                                                 ...                ...   
9573                               Welcome to Daisyland          2053.3320   
9574                                     Werewolf Fever          8315.8326   
9575                                       Without Name          8770.5000   
9576  You're So Cool, Brewster! The Story of Fright ...                NaN   
9577                                       Zombi Horror                NaN   

                                                                        \
Date    2020-05    2020-06    2020-07    2020-08    2020-09    2020-10   
0           NaN        NaN        NaN        NaN        NaN        NaN   
1           NaN        NaN        NaN        NaN        NaN        NaN   
2           NaN        NaN        NaN        NaN        NaN        NaN   
3           NaN        NaN        NaN        NaN        NaN        NaN   
4           NaN        NaN        NaN        NaN        NaN        NaN   
...         ...        ...        ...        ...        ...        ...   
9573  1233.1656   678.3324  1002.9990   561.1650   198.6654   993.1650   
9574  6783.9996        NaN        NaN        NaN        NaN        NaN   
9575  3796.6662  1918.8324  6371.4996  4564.3326  2009.9994  3216.8328   
9576        NaN        NaN        NaN        NaN        NaN        NaN   
9577        NaN        NaN        NaN        NaN        NaN        NaN   

                                                                           \
Date   2020-11   2020-12   2021-01  2021-02   2021-03   2021-04   2021-05   
0          NaN       NaN       NaN      NaN       NaN       NaN       NaN   
1          NaN       NaN       NaN      NaN       NaN       NaN       NaN   
2          NaN       NaN       NaN      NaN       NaN       NaN       NaN   
3          NaN       NaN       NaN      NaN       NaN       NaN       NaN   
4          NaN       NaN       NaN      NaN       NaN       NaN       NaN   
...        ...       ...       ...      ...       ...       ...       ...   
9573  268.9986  506.1654  395.6652  86.9988  337.6656  274.3314  142.6644   
9574       NaN       NaN       NaN      NaN       NaN       NaN       NaN   
9575       NaN       NaN       NaN      NaN       NaN       NaN       NaN   
9576       NaN       NaN       NaN      NaN       NaN       NaN       NaN   
9577       NaN       NaN       NaN      NaN       NaN       NaN       NaN   

                                                                              \
Date   2021-06   2021-07   2021-08   2021-09   2021-10    2021-11    2021-12   
0          NaN       NaN       NaN       NaN       NaN     0.8000  1634.4200   
1          NaN       NaN       NaN    3.1662    2.6664   322.2432   606.7564   
2          NaN       NaN    1.6662  210.1

In [122]:

CMSS.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD+['Metric']
CMSSM.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD+['Metric']
RPP.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD+['Metric']


In [123]:
CM=pd.concat([CMSS,CMSSM,RPP])

In [124]:
CM

,Subscription,ID,TV/MOVIE,Title,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Metric
0,CONtv,1000000000019,MOVIE,The Astrologer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.800000,1634.420000,1883.400000,1513.960000,1453.460000,1082.100000,654.71000,251.340000,729.080000,354.82000,332.250000,237.700000,11.810000,106.420000,337.440000,248.360000,492.120000,Minutes Streamed
1,CONtv,1000000000029,MOVIE,Psychos In Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.166200,2.666400,322.243200,606.756400,573.526600,1061.833000,863.539600,396.020000,388.59660,624.126600,846.180000,518.01000,300.710000,457.340000,440.240000,512.520000,245.900000,792.196600,639.426400,Minutes Streamed
2,CONtv,1000000000180,MOVIE,Guardian Of The Highlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666200,210.169800,87.732800,10.300000,1.666200,157.776600,NaN,77.890000,2.430000,NaN,280.240000,32.820000,393.36000,407.450000,334.380000,102.000000,196.650000,260.760000,285.646400,364.546200,Minutes Streamed
3,CONtv,1000000000196,MOVIE,The Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.510000,279.470000,514.330000,393.410000,118.170000,8.950000,190.01000,16.880000,115.170000,94.70000,0.150000,6.160000,168.230000,81.690000,197.840000,163.680000,104.630000,Minutes Streamed
4,CONtv,1000000000197,MOVIE,Bad Man's River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333000,597.166200,1290.176400,419.320000,571.719800,436.889800,652.556600,370.743200,207.00000,720.712800,847.960000,952.33980,1214.299800,99.166200,186.166200,416.616400,838.512800,531.806200,1765.422800,Minutes Streamed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,Screambox,WELCOMETODAISYLAND,TV,Welcome to Daisyland,0.542056,0.464455,0.463918,0.578125,0.497326,0.22,0.388889,0.255102,0.349754,0.319048,0.129187,0.336538,0.242857,0.194444,0.284404,0.30315,0.372881,0.274667,0.260417,0.324561,0.240741,0.159204,0.191693,0.214506,0.081897,0.21733,0.198034,0.339943,0.31044,0.154967,0.340702,0.336761,0.291415,0.318236,0.174432,0.254023,Percentile Rank (Min)
9574,Screambox,WEREWOLFFEVER,MOVIE,Werewolf Fever,0.827103,0.848341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentile Rank (Min)
9575,Screambox,WITHOUTNAME,MOVIE,Without Name,0.845794,0.739336,0.680412,0.890625,0.871658,0.78,0.661616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentile Rank (Min)
9576,Screambox,YOURESOCOOLBREWSTER,TV,"You're So Cool, Brewster! The Story of Fright ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.594278,0.485861,0.540508,0.466031,0.695341,0.803448,Percentile Rank (Min)


In [125]:
#Average
CM['Average']=CM.loc[:,'2020-04':'2023-03'].mean(axis=1)

#Average Monthly Difference
CM['F']=CM.loc[:,'2020-04':'2023-03'].fillna(method='bfill',axis=1).iloc[:,0]
CM['L']=CM.loc[:,'2020-04':'2023-03'].fillna(method='ffill',axis=1).iloc[:,-1]
CM['C']=CM.loc[:,'2020-04':'2023-03'].count(axis=1)

CM['Trend']= (CM['L']-CM['F'])/(CM['C']-1)

In [126]:
CM=CM.loc[:,('Subscription','ID','TV/MOVIE','Title','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12',
             '2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12',
             '2022-01','2022-02','2022-03','2022-04','2022-05','2022-06','2022-07','2022-08','2022-09','2022-10','2022-11','2022-12','2023-01','2023-02','2023-03','Metric','Average','Trend')]

In [127]:
CM.head(-100)

,Subscription,ID,TV/MOVIE,Title,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Metric,Average,Trend
0,CONtv,1000000000019,MOVIE,The Astrologer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.800000,1634.420000,1883.400000,1513.960000,1453.460000,1082.100000,654.710000,251.340000,729.080000,354.820000,332.250000,237.700000,11.810000,106.420000,337.440000,248.360000,492.120000,Minutes Streamed,666.128824,30.707500
1,CONtv,1000000000029,MOVIE,Psychos In Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.166200,2.666400,322.243200,606.756400,573.526600,1061.833000,863.539600,396.020000,388.596600,624.126600,846.180000,518.010000,300.710000,457.340000,440.240000,512.520000,245.900000,792.196600,639.426400,Minutes Streamed,504.999874,35.347789
2,CONtv,1000000000180,MOVIE,Guardian Of The Highlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666200,210.169800,87.732800,10.300000,1.666200,157.776600,NaN,77.890000,2.430000,NaN,280.240000,32.820000,393.360000,407.450000,334.380000,102.000000,196.650000,260.760000,285.646400,364.546200,Minutes Streamed,178.193567,21.345882
3,CONtv,1000000000196,MOVIE,The Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.510000,279.470000,514.330000,393.410000,118.170000,8.950000,190.010000,16.880000,115.170000,94.700000,0.150000,6.160000,168.230000,81.690000,197.840000,163.680000,104.630000,Minutes Streamed,144.410588,6.445000
4,CONtv,1000000000197,MOVIE,Bad Man's River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333000,597.166200,1290.176400,419.320000,571.719800,436.889800,652.556600,370.743200,207.000000,720.712800,847.960000,952.339800,1214.299800,99.166200,186.166200,416.616400,838.512800,531.806200,1765.422800,Minutes Streamed,638.153053,97.727211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9473,Screambox,SBOX100000240,MOVIE,The Killing of Jacob Marr,0.598131,0.687204,0.592784,0.697917,0.620321,0.490,0.590909,0.586735,0.522167,0.514286,0.492823,0.572115,0.366667,0.476852,0.587156,0.484252,0.291525,0.738667,0.718750,0.659649,0.622896,0.447761,0.504792,0.520062,0.458333,0.365057,0.466292,0.426346,0.645604,0.737748,0.842653,0.867609,0.808948,0.810489,0.796894,0.829885,Percentile Rank (Min),0.595563,0.006622
9474,Screambox,SBOX100000243,MOVIE,The Pale Man,0.523364,0.445498,0.747423,0.588542,0.582888,0.840,0.636364,0.515306,0.497537,0.595238,0.612440,0.528846,0.557143,0.509259,0.665138,0.311024,0.247458,0.560000,0.537500,0.692982,0.670034,0.631841,0.471246,0.387346,0.426724,0.237216,0.355337,0.215297,0.337912,0.317881,0.224967,0.158098,0.258767,0.319428,0.370370,0.354023,Percentile Rank (Min),0.470290,-0.004838
9475,Screambox,SBOX100000245,MOVIE,The Portal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.460417,0.850877,0.882155,0.862355,0.843450,0.824074,0.811782,0.838068,0.849719,0.820113,0.820055,0.818543,0.843953,0.859897,0.887545,0.802145,0.808841,0.748276,Percentile Rank (Min),0.812904,0.016933
9476,Screambox,SBOX100000246,MOVIE,The Real Haunted Mansion,0.056075,0.004739,0.103093,NaN,NaN,0.055,0.297980,0.188776,0.147783,0.042857,0.095694,0.057692,0.176190,0.189815,0.110092,0.153543,NaN,0.469333,0.520833,0.447368,0.358586,0.280265,0.314696,0.254630,0.204023,0.376420,0.157303,0.368272,0.424451,0.344371,0.248375,0.152956,0.198307,0.222884,0.339307,0.370115,Percentile Rank (Min),0.234298,0.009814


In [128]:
Subs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              180 non-null    int64  
 1   Month(num)        180 non-null    int64  
 2   Month             180 non-null    object 
 3   Subscription      180 non-null    object 
 4   Amazon Paid Subs  180 non-null    int64  
 5   Roku Paid Subs    180 non-null    int64  
 6   Billing Amount    180 non-null    float64
 7   Total Paid Subs   180 non-null    int64  
 8   Revenue           180 non-null    float64
dtypes: float64(2), int64(5), object(2)
memory usage: 14.1+ KB


# Adding Total Minutes and Streams to Sub table

In [129]:
C3= Content2.loc[:,('Subscription','Year','Month','Total Min Streamed','Total Streams','Amzn Min Streamed','Amzn Streams','Roku Min Streamed','Roku Streams')]
C3['Month']=C3['Month'].astype(int)
C3= C3.groupby(by=['Subscription','Year','Month',])[['Total Min Streamed','Total Streams','Amzn Min Streamed','Amzn Streams','Roku Min Streamed','Roku Streams']].agg(sum).reset_index()
Subs= pd.merge(Subs,C3, left_on=('Subscription','Year','Month(num)'), right_on=('Subscription','Year','Month'), how='left')
Subs.rename(columns={'Month_x':'Month'}, inplace=True)
# Subs= Subs.loc[:,('Subscription','Year','Month(num)','Month','Amazon Paid Subs','Amzn Min Streamed','Amzn Streams',
#                  'Roku Paid Subs','Roku Min Streamed','Roku Streams','Billing Amount','Total Paid Subs','Revenue','Total Min Streamed','Total Streams')]

Subs['Amzn Min/Stream']= Subs['Amzn Min Streamed']/Subs['Amzn Streams']
Subs['Roku Min/Stream']= Subs['Roku Min Streamed']/Subs['Roku Streams']
Subs['Avg Min/Stream']= Subs['Total Min Streamed']/Subs['Total Streams']

Subs['Amzn Min/Sub']= Subs['Amzn Min Streamed']/Subs['Amazon Paid Subs']
Subs['Roku Min/Sub']= Subs['Roku Min Streamed']/Subs['Roku Paid Subs']
Subs['Min/Sub']= Subs['Total Min Streamed']/Subs['Total Paid Subs']

In [130]:
Content2

,Subscription,Year,Month,Month Abv,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Roku Min Streamed,Roku Streams,Roku Min/Stream,Total Min Streamed,Percentile Rank (Min),Total Streams,Min/Stream,Content Month Rev,Date
0,CONtv,2020,04,Apr,CEG10018117,MOVIE,100 Below Zero,89.02,2.0,44.510000,1015.9998,18.0,56.444433,1105.0198,0.614765,20.0,55.250990,7.566223,2020-04
1,CONtv,2020,04,Apr,CEG10018177,MOVIE,100 Ghost Street: The Return of Richard Speck,1.08,1.0,1.080000,845.4996,15.0,56.366640,846.5796,0.577181,16.0,52.911225,5.796647,2020-04
2,CONtv,2020,04,Apr,CEG10018118,MOVIE,100 Million BC,1016.65,19.0,53.507895,1116.3330,26.0,42.935885,2132.9830,0.718121,45.0,47.399622,14.604829,2020-04
3,CONtv,2020,04,Apr,CEG10018116,MOVIE,11/11/11,292.05,7.0,41.721429,423.0000,9.0,47.000000,715.0500,0.551678,16.0,44.690625,4.896046,2020-04
4,CONtv,2020,04,Apr,CEG10020241,MOVIE,13 Demons,716.62,23.0,31.157391,0.0000,0.0,NaN,716.6200,0.553020,23.0,31.157391,4.906796,2020-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165710,Screambox,2023,03,Mar,THEDEADZONE,TV,The Dead Zone,10868.09,394.0,27.583985,452286.0264,12584.0,35.941356,463154.1164,0.995402,12978.0,35.687634,16350.410936,2023-03
165711,Screambox,2023,03,Mar,THEISLAND,TV,The Island,45435.87,1651.0,27.520212,140363.9496,4662.0,30.108097,185799.8196,0.989655,6313.0,29.431304,6559.163127,2023-03
165712,Screambox,2023,03,Mar,TOXICCRUSADERS,TV,Toxic Crusaders,7324.44,480.0,15.259250,11615.4558,819.0,14.182486,18939.8958,0.862069,1299.0,14.580366,668.622103,2023-03
165713,Screambox,2023,03,Mar,WELCOMETODAISYLAND,TV,Welcome to Daisyland,0.00,0.0,NaN,432.9990,90.0,4.811100,432.9990,0.254023,90.0,4.811100,15.285866,2023-03


In [131]:
C3

,Subscription,Year,Month,Total Min Streamed,Total Streams,Amzn Min Streamed,Amzn Streams,Roku Min Streamed,Roku Streams
0,CONtv,2020,4,1.993045e+07,787884.0,2494913.23,111911.0,1.743553e+07,675973.0
1,CONtv,2020,5,2.214802e+07,805018.0,2569089.38,115117.0,1.957893e+07,689901.0
2,CONtv,2020,6,1.610129e+07,570768.0,2297150.60,101963.0,1.380414e+07,468805.0
3,CONtv,2020,7,1.498874e+07,529738.0,2832018.33,117724.0,1.215672e+07,412014.0
4,CONtv,2020,8,1.290963e+07,467538.0,2205753.84,96344.0,1.070387e+07,371194.0
5,CONtv,2020,9,1.523947e+07,544174.0,2164008.28,92631.0,1.307546e+07,451543.0
6,CONtv,2020,10,1.387524e+07,486382.0,2365955.76,96221.0,1.150928e+07,390161.0
7,CONtv,2020,11,1.376655e+07,494931.0,2832501.78,128918.0,1.093405e+07,366013.0
8,CONtv,2020,12,1.094688e+07,389380.0,2269143.46,93255.0,8.677735e+06,296125.0
9,CONtv,2021,1,9.859645e+06,399047.0,2377579.88,94629.0,7.482066e+06,304418.0


In [132]:
Subs['Amzn Rev']=Subs['Amazon Paid Subs']*Subs['Billing Amount']
Subs['Roku Rev']=Subs['Roku Paid Subs']*Subs['Billing Amount']

Subs= Subs.loc[:,('Subscription','Year','Month(num)','Month','Billing Amount',
                  'Amazon Paid Subs','Amzn Rev','Amzn Min Streamed','Amzn Streams','Amzn Min/Stream','Amzn Min/Sub',
                  'Roku Paid Subs','Roku Rev','Roku Min Streamed','Roku Streams','Roku Min/Stream','Roku Min/Sub',
                  'Total Paid Subs','Revenue','Total Min Streamed','Total Streams','Avg Min/Stream','Min/Sub')]

In [133]:
Subs.head(200)

,Subscription,Year,Month(num),Month,Billing Amount,Amazon Paid Subs,Amzn Rev,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Amzn Min/Sub,Roku Paid Subs,Roku Rev,Roku Min Streamed,Roku Streams,Roku Min/Stream,Roku Min/Sub,Total Paid Subs,Revenue,Total Min Streamed,Total Streams,Avg Min/Stream,Min/Sub
0,CONtv,2020,4,Apr,4.99,26140,130438.60,2494913.23,111911.0,22.293727,95.444270,1208,6027.92,1.743553e+07,675973.0,25.793239,14433.388537,27348,136466.52,1.993045e+07,787884.0,25.296169,728.771632
1,CONtv,2020,5,May,4.99,27697,138208.03,2569089.38,115117.0,22.317202,92.756955,2279,11372.21,1.957893e+07,689901.0,28.379336,8591.019015,29976,149580.24,2.214802e+07,805018.0,27.512455,738.858477
2,CONtv,2020,6,Jun,4.99,32540,162374.60,2297150.60,101963.0,22.529257,70.594671,2613,13038.87,1.380414e+07,468805.0,29.445386,5282.871892,35153,175413.47,1.610129e+07,570768.0,28.209877,458.034730
3,CONtv,2020,7,Jul,4.99,34989,174595.11,2832018.33,117724.0,24.056423,80.940248,2431,12130.69,1.215672e+07,412014.0,29.505606,5000.708696,37420,186725.80,1.498874e+07,529738.0,28.294631,400.554280
4,CONtv,2020,8,Aug,4.99,35142,175358.58,2205753.84,96344.0,22.894564,62.766884,2232,11137.68,1.070387e+07,371194.0,28.836332,4795.642175,37374,186496.26,1.290963e+07,467538.0,27.611931,345.417327
5,CONtv,2020,9,Sep,4.99,34955,174425.45,2164008.28,92631.0,23.361599,61.908405,2009,10024.91,1.307546e+07,451543.0,28.957281,6508.440839,36964,184450.36,1.523947e+07,544174.0,28.004767,412.278593
6,CONtv,2020,10,Oct,4.99,35633,177808.67,2365955.76,96221.0,24.588767,66.397883,1844,9201.56,1.150928e+07,390161.0,29.498808,6241.477511,37477,187010.23,1.387524e+07,486382.0,28.527454,370.233484
7,CONtv,2020,11,Nov,4.99,36933,184295.67,2832501.78,128918.0,21.971344,76.692979,1833,9146.67,1.093405e+07,366013.0,29.873380,5965.109269,38766,193442.34,1.376655e+07,494931.0,27.815083,355.119101
8,CONtv,2020,12,Dec,4.99,36900,184131.00,2269143.46,93255.0,24.332673,61.494403,1947,9715.53,8.677735e+06,296125.0,29.304297,4456.977437,38847,193846.53,1.094688e+07,389380.0,28.113613,281.794695
9,CONtv,2021,1,Jan,4.99,37155,185403.45,2377579.88,94629.0,25.125277,63.990846,2229,11122.71,7.482066e+06,304418.0,24.578263,3356.691583,39384,196526.16,9.859645e+06,399047.0,24.707980,250.346471


# REVENUE

In [134]:
Rev= Content2.loc[:,('Subscription','Year','Month','ID','Date','TV/MOVIE','Title','Content Month Rev')]

In [135]:
Rev1= Rev.pivot(index=['Subscription','ID','TV/MOVIE','Title'], columns='Date', values=['Content Month Rev']).reset_index()

In [136]:
Rev1.columns=['Subscription','ID','TV/MOVIE','Title']+ColumnD

Rev1=Rev1.loc[:,('Subscription','ID','TV/MOVIE','Title','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12',
             '2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12',
             '2022-01','2022-02','2022-03','2022-04','2022-05','2022-06','2022-07','2022-08','2022-09','2022-10','2022-11','2022-12','2023-01','2023-02','2023-03')]

#Average
Rev1['Average']=Rev1.loc[:,'2020-04':'2023-03'].mean(axis=1)

#Average Monthly Difference
Rev1['F']=Rev1.loc[:,'2020-04':'2023-03'].fillna(method='bfill',axis=1).iloc[:,0]
Rev1['L']=Rev1.loc[:,'2020-04':'2023-03'].fillna(method='ffill',axis=1).iloc[:,-1]
Rev1['C']=Rev1.loc[:,'2020-04':'2023-03'].count(axis=1)

Rev1['Trend']= (Rev1['L']-Rev1['F'])/(Rev1['C']-1)



Rev1=Rev1.loc[:,('Subscription','ID','TV/MOVIE','Title','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12',
             '2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12',
             '2022-01','2022-02','2022-03','2022-04','2022-05','2022-06','2022-07','2022-08','2022-09','2022-10','2022-11','2022-12','2023-01','2023-02','2023-03','Average','Trend')]

In [137]:
Rev1

,Subscription,ID,TV/MOVIE,Title,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Average,Trend
0,CONtv,1000000000019,MOVIE,The Astrologer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014874,35.507753,31.604657,23.711389,21.538645,15.009008,1.755791,0.490172,1.279526,2.536042,4.645040,4.929616,0.041680,0.837066,3.966909,4.184444,6.115627,9.304014,0.381297
1,CONtv,1000000000029,MOVIE,Psychos In Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065069,0.046843,5.991302,13.181775,9.624143,16.630251,12.796687,5.492900,1.042132,1.217194,1.485035,3.702427,4.204093,9.484689,1.553692,4.031319,2.890775,13.347167,7.946219,6.038616,0.437842
2,CONtv,1000000000180,MOVIE,Guardian Of The Highlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022621,4.319257,1.541264,0.191503,0.036198,2.647592,NaN,1.154242,0.033705,NaN,0.546534,0.057599,2.811503,5.696378,6.934644,0.359978,1.546786,3.065467,4.812656,4.530254,2.239343,0.265155
3,CONtv,1000000000196,MOVIE,The Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028075,6.071482,8.630787,6.161522,1.751147,0.124139,0.509566,0.032920,0.202122,0.676859,0.002097,0.127751,0.593716,0.642548,2.325786,2.757730,1.300248,1.878735,0.079511
4,CONtv,1000000000197,MOVIE,Bad Man's River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130151,10.490839,23.987585,9.109721,9.593824,6.842495,9.670156,5.142304,0.555129,1.405560,1.488159,6.806758,16.976588,2.056589,0.657016,3.276972,9.857469,8.960031,21.939094,7.839286,1.211608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,Screambox,WELCOMETODAISYLAND,TV,Welcome to Daisyland,32.395970,20.035365,19.292995,22.000019,10.858643,5.286587,20.26157,5.830574,5.72241,8.342684,2.81162,5.165193,13.859854,6.321581,7.919916,25.306872,21.557545,11.054143,17.392220,25.040735,2.163035,1.553928,2.235634,2.660140,0.789280,2.040818,1.400696,6.676877,3.942059,1.602328,5.586020,9.031688,14.274350,19.659808,9.379428,15.285866,10.687179,-0.488860
9574,Screambox,WEREWOLFFEVER,MOVIE,Werewolf Fever,131.201122,110.220321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.710721,-20.980801
9575,Screambox,WITHOUTNAME,MOVIE,Without Name,138.374532,61.684816,54.575049,139.753988,88.320652,53.487103,65.62664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.974683,-12.124649
9576,Screambox,YOURESOCOOLBREWSTER,TV,"You're So Cool, Brewster! The Story of Fright ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.129292,19.509882,68.845565,48.143616,242.567919,433.759587,139.325977,82.126059


In [138]:
Rev1.rename(columns={'Average':'Roku+Amzn monthly Rev Avg'},inplace=True)
Rev1['All Platforms monthly Rev Avg (10/8 estimate)']= Rev1['Roku+Amzn monthly Rev Avg']*(10/8)

In [139]:
#Add 0 in front of single digits
TotMRevM['Month'] = TotMRevM['Month'].astype(str).str.zfill(2)

In [140]:
TotMRevM['Date']= TotMRevM['Year'].astype(str)+'-'+TotMRevM['Month'].astype(str)
TotMRevM= TotMRevM.pivot(index=['Subscription'], columns='Date', values=['Tot Sub Month Rev']).reset_index()

In [141]:
TotMRevM

Subscription Tot Sub Month Rev                                   \
Date                        2020-04    2020-05    2020-06    2020-07   
0           CONtv         136466.52  149580.24  175413.47  186725.80   
1        Docurama          52210.37   56756.26   65089.56   68183.36   
2            Dove         132389.69  147564.28  167938.45  173996.31   
3          Fandor         178532.55  193175.85  224461.44  224058.45   
4       Screambox          29715.45   39071.70   47884.04   49321.16   

                                                                        \
Date    2020-08    2020-09    2020-10    2020-11    2020-12    2021-01   
0     186496.26  184450.36  187010.23  193442.34  193846.53  196526.16   
1      69326.07   74226.25   79745.19   79790.10   79770.14   81327.02   
2     184415.43  185143.97  187673.90  189999.24  194155.91  196136.94   
3     222430.53  222175.17  226815.54  229967.64  231846.93  236204.01   
4      48732.34   49430.94   55149.48   54076.63   58956.85   68602.52   

                                                                        \
Date    2021-02    2021-03    2021-04    2021-05    2021-06    2021-07   
0     200897.40  207484.20  225248.60  235478.10  241645.74  251705.58   
1      82564.54   84949.76   88278.09   97060.49  102744.10  111291.97   
2     198078.05  198731.74  202379.43  206541.09  207948.27  211920.31   
3     239100.75  242955.09  248497.20  254306.64  257095.65  263032.77   
4      69346.03   68662.40   70363.99   72963.78   75304.09   80778.12   

                                                                        \
Date    2021-08    2021-09    2021-10    2021-11    2021-12    2022-01   
0     256675.62  332438.79  336490.67  321046.62  317354.02  328202.28   
1     122928.65  156920.53  158652.06  155867.64  155468.44  158662.04   
2     217030.07  250393.21  253467.05  260233.49  284584.69  295687.44   
3     267158.43  320688.27  320540.64  317472.33  315668.85  325919.16   
4      84405.85  116037.46  130538.40  142958.51  141002.43  150109.18   

                                                                        \
Date    2022-02    2022-03    2022-04    2022-05    2022-06    2022-07   
0     327982.72  328361.96  335911.83  343067.49  343850.92  347019.57   
1     163302.74  170463.39  176082.13  179600.08  183152.96  189789.66   
2     288476.89  284090.68  289789.26  288566.71  290093.65  287873.10   
3     323932.14  324905.70  330208.41  328508.67  327714.66  331445.31   
4     161571.21  179121.04  187434.38  183731.80  181196.88  180977.32   

                                                                         \
Date    2022-08    2022-09    2022-10    2022-11     2022-12    2023-01   
0     348516.57  334709.24  314050.64  304100.58   298367.07  297458.89   
1     193741.74  192000.23  187349.55  189624.99   192015.20  196461.29   
2     288911.02  297284.24  300542.71  317034.66   328496.69  344459.70   
3     337019.34  333599.91  329941.08  334210.38   336767.97  346351.95   
4     227648.79  257843.28  310931.89  914716.90  1098229.14  991912.20   

                            
Date    2023-02    2023-03  
0     297967.87  297443.92  
1     196231.75  198392.42  
2     349220.16  355063.45  
3     345613.80  342633.27  
4     943663.89  920640.03

In [142]:
TotMRevM.columns=['Subscription','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12',
                         '2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12',
                         '2022-01','2022-02','2022-03','2022-04','2022-05','2022-06','2022-07','2022-08','2022-09','2022-10','2022-11','2022-12',
                         '2023-01','2023-02','2023-03']

In [143]:
TotMRevM.loc[-1]=['Number',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36]

In [144]:
TotMRevM

,Subscription,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,CONtv,136466.52,149580.24,175413.47,186725.80,186496.26,184450.36,187010.23,193442.34,193846.53,196526.16,200897.40,207484.20,225248.60,235478.10,241645.74,251705.58,256675.62,332438.79,336490.67,321046.62,317354.02,328202.28,327982.72,328361.96,335911.83,343067.49,343850.92,347019.57,348516.57,334709.24,314050.64,304100.58,298367.07,297458.89,297967.87,297443.92
1,Docurama,52210.37,56756.26,65089.56,68183.36,69326.07,74226.25,79745.19,79790.10,79770.14,81327.02,82564.54,84949.76,88278.09,97060.49,102744.10,111291.97,122928.65,156920.53,158652.06,155867.64,155468.44,158662.04,163302.74,170463.39,176082.13,179600.08,183152.96,189789.66,193741.74,192000.23,187349.55,189624.99,192015.20,196461.29,196231.75,198392.42
2,Dove,132389.69,147564.28,167938.45,173996.31,184415.43,185143.97,187673.90,189999.24,194155.91,196136.94,198078.05,198731.74,202379.43,206541.09,207948.27,211920.31,217030.07,250393.21,253467.05,260233.49,284584.69,295687.44,288476.89,284090.68,289789.26,288566.71,290093.65,287873.10,288911.02,297284.24,300542.71,317034.66,328496.69,344459.70,349220.16,355063.45
3,Fandor,178532.55,193175.85,224461.44,224058.45,222430.53,222175.17,226815.54,229967.64,231846.93,236204.01,239100.75,242955.09,248497.20,254306.64,257095.65,263032.77,267158.43,320688.27,320540.64,317472.33,315668.85,325919.16,323932.14,324905.70,330208.41,328508.67,327714.66,331445.31,337019.34,333599.91,329941.08,334210.38,336767.97,346351.95,345613.80,342633.27
4,Screambox,29715.45,39071.70,47884.04,49321.16,48732.34,49430.94,55149.48,54076.63,58956.85,68602.52,69346.03,68662.40,70363.99,72963.78,75304.09,80778.12,84405.85,116037.46,130538.40,142958.51,141002.43,150109.18,161571.21,179121.04,187434.38,183731.80,181196.88,180977.32,227648.79,257843.28,310931.89,914716.90,1098229.14,991912.20,943663.89,920640.03
-1,Number,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00,13.00,14.00,15.00,16.00,17.00,18.00,19.00,20.00,21.00,22.00,23.00,24.00,25.00,26.00,27.00,28.00,29.00,30.00,31.00,32.00,33.00,34.00,35.00,36.00


In [145]:
def linear_regression_int(x, y):     
    N = len(x)
    x_mean = x.mean()
    y_mean = y.mean()
    
    B1_num = ((x - x_mean) * (y - y_mean)).sum()
    B1_den = ((x - x_mean)**2).sum()
    Slope = B1_num / B1_den
    
    B0 = y_mean - (Slope*x_mean)
    
    
    return (B0)

In [146]:
def linear_regression_slope(x, y):     
    N = len(x)
    x_mean = x.mean()
    y_mean = y.mean()
    
    B1_num = ((x - x_mean) * (y - y_mean)).sum()
    B1_den = ((x - x_mean)**2).sum()
    
    
    Slope = B1_num / B1_den
    
    B0 = y_mean - (Slope*x_mean)
    
    
    return (Slope)

In [147]:
TotMRevM

,Subscription,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,CONtv,136466.52,149580.24,175413.47,186725.80,186496.26,184450.36,187010.23,193442.34,193846.53,196526.16,200897.40,207484.20,225248.60,235478.10,241645.74,251705.58,256675.62,332438.79,336490.67,321046.62,317354.02,328202.28,327982.72,328361.96,335911.83,343067.49,343850.92,347019.57,348516.57,334709.24,314050.64,304100.58,298367.07,297458.89,297967.87,297443.92
1,Docurama,52210.37,56756.26,65089.56,68183.36,69326.07,74226.25,79745.19,79790.10,79770.14,81327.02,82564.54,84949.76,88278.09,97060.49,102744.10,111291.97,122928.65,156920.53,158652.06,155867.64,155468.44,158662.04,163302.74,170463.39,176082.13,179600.08,183152.96,189789.66,193741.74,192000.23,187349.55,189624.99,192015.20,196461.29,196231.75,198392.42
2,Dove,132389.69,147564.28,167938.45,173996.31,184415.43,185143.97,187673.90,189999.24,194155.91,196136.94,198078.05,198731.74,202379.43,206541.09,207948.27,211920.31,217030.07,250393.21,253467.05,260233.49,284584.69,295687.44,288476.89,284090.68,289789.26,288566.71,290093.65,287873.10,288911.02,297284.24,300542.71,317034.66,328496.69,344459.70,349220.16,355063.45
3,Fandor,178532.55,193175.85,224461.44,224058.45,222430.53,222175.17,226815.54,229967.64,231846.93,236204.01,239100.75,242955.09,248497.20,254306.64,257095.65,263032.77,267158.43,320688.27,320540.64,317472.33,315668.85,325919.16,323932.14,324905.70,330208.41,328508.67,327714.66,331445.31,337019.34,333599.91,329941.08,334210.38,336767.97,346351.95,345613.80,342633.27
4,Screambox,29715.45,39071.70,47884.04,49321.16,48732.34,49430.94,55149.48,54076.63,58956.85,68602.52,69346.03,68662.40,70363.99,72963.78,75304.09,80778.12,84405.85,116037.46,130538.40,142958.51,141002.43,150109.18,161571.21,179121.04,187434.38,183731.80,181196.88,180977.32,227648.79,257843.28,310931.89,914716.90,1098229.14,991912.20,943663.89,920640.03
-1,Number,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00,13.00,14.00,15.00,16.00,17.00,18.00,19.00,20.00,21.00,22.00,23.00,24.00,25.00,26.00,27.00,28.00,29.00,30.00,31.00,32.00,33.00,34.00,35.00,36.00


In [148]:
RevP= TotMRevM.set_index('Subscription').T
RevP.reset_index(inplace=True)
RevP.columns=['Date','CONtv','Docurama','Dove','Fandor','Screambox','Number']

In [149]:
RevP

,Date,CONtv,Docurama,Dove,Fandor,Screambox,Number
0,2020-04,136466.52,52210.37,132389.69,178532.55,29715.45,1.0
1,2020-05,149580.24,56756.26,147564.28,193175.85,39071.70,2.0
2,2020-06,175413.47,65089.56,167938.45,224461.44,47884.04,3.0
3,2020-07,186725.80,68183.36,173996.31,224058.45,49321.16,4.0
4,2020-08,186496.26,69326.07,184415.43,222430.53,48732.34,5.0
5,2020-09,184450.36,74226.25,185143.97,222175.17,49430.94,6.0
6,2020-10,187010.23,79745.19,187673.90,226815.54,55149.48,7.0
7,2020-11,193442.34,79790.10,189999.24,229967.64,54076.63,8.0
8,2020-12,193846.53,79770.14,194155.91,231846.93,58956.85,9.0
9,2021-01,196526.16,81327.02,196136.94,236204.01,68602.52,10.0


In [150]:
linear_regression_slope(RevP['Number'],RevP['Fandor'])

4700.619513513515

In [151]:
RevPrj= pd.DataFrame([
    ['CONtv',linear_regression_int(RevP['Number'],RevP['CONtv']),linear_regression_slope(RevP['Number'],RevP['CONtv'])],
    ['Docurama',linear_regression_int(RevP['Number'],RevP['Docurama']),linear_regression_slope(RevP['Number'],RevP['Docurama'])],
    ['Dove',linear_regression_int(RevP['Number'],RevP['Dove']),linear_regression_slope(RevP['Number'],RevP['Dove'])],
    ['Fandor',linear_regression_int(RevP['Number'],RevP['Fandor']),linear_regression_slope(RevP['Number'],RevP['Fandor'])],
    ['Screambox',linear_regression_int(RevP['Number'],RevP['Screambox']),linear_regression_slope(RevP['Number'],RevP['Screambox'])]
    
], 
                     columns=['Subscription','intercept','Slope',])

In [152]:
RevPrj

,Subscription,intercept,Slope
0,CONtv,163422.452476,5525.865632
1,Docurama,44702.669460,4775.862852
2,Dove,140936.847143,5679.557632
3,Fandor,197342.885667,4700.619514
4,Screambox,-168909.987159,21807.491949


In [153]:
Mon1=RevP.iloc[-1,6]+1
Year1=RevP.iloc[-1,6]+12
Year2=RevP.iloc[-1,6]+24
Year3=RevP.iloc[-1,6]+36
Year4=RevP.iloc[-1,6]+48


ProjTable=pd.DataFrame([
    [RevP['Number'].iloc[-2], RevP.iloc[-2]['Date'], RevP.iloc[-2]['CONtv'], RevP.iloc[-2]['Docurama'], RevP.iloc[-2]['Dove'], RevP.iloc[-2]['Fandor'], RevP.iloc[-2]['Screambox'] ],
    [RevP['Number'].iloc[-1], RevP.iloc[-1]['Date'], RevP.iloc[-1]['CONtv'], RevP.iloc[-1]['Docurama'], RevP.iloc[-1]['Dove'], RevP.iloc[-1]['Fandor'], RevP.iloc[-1]['Screambox']  ],
    # 1 Month later
    [Mon1 , '3-2023',   (Mon1*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Mon1*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    # 1 Year Later
   [Year1 , '4-2023',   (Year1*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year1*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    # 2 Years Later
    [Year2 , '4-2024',   (Year2*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year2*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    #3 Years Later
    [Year3 , '4-2025',   (Year3*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year3*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]],
    #4 Years Later
    [Year4 , '4-2026',   (Year4*RevPrj.loc[RevPrj['Subscription']=='CONtv','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='CONtv','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Docurama','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Docurama','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Dove','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Dove','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Fandor','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Fandor','intercept'].values[0],
                     (Year4*RevPrj.loc[RevPrj['Subscription']=='Screambox','Slope'].values[0]) + RevPrj.loc[RevPrj['Subscription']=='Screambox','intercept'].values[0]]
],
                       columns=['Number','Date','CONtv','Docurama','Dove','Fandor','Screambox'])

In [154]:
(Mon1*RevPrj.iloc[0]['Slope']) + RevPrj.loc[RevPrj['Subscription']=='CONtv']['intercept'].values[0]

367879.4808571429

In [155]:
ProjTable

,Number,Date,CONtv,Docurama,Dove,Fandor,Screambox
0,35.0,2023-02,297967.870000,196231.750000,349220.160000,345613.800000,9.436639e+05
1,36.0,2023-03,297443.920000,198392.420000,355063.450000,342633.270000,9.206400e+05
2,37.0,3-2023,367879.480857,221409.594984,351080.479524,371265.807667,6.379672e+05
3,48.0,4-2023,428664.002808,273944.086356,413555.613475,422972.622315,8.778496e+05
4,60.0,4-2024,494974.390391,331254.440580,481710.305058,479380.056477,1.139540e+06
5,72.0,4-2025,561284.777974,388564.794804,549864.996641,535787.490640,1.401229e+06
6,84.0,4-2026,627595.165557,445875.149028,618019.688224,592194.924802,1.662919e+06


# Subs Projections

In [156]:
TotMSubs= TotMRevM.loc[:,:].copy()

In [157]:
TotMSubs.loc[TotMSubs['Subscription']=='CONtv','2020-04':]=TotMSubs.loc[TotMSubs['Subscription']=='CONtv','2020-04':].div(4.99)
TotMSubs.loc[TotMSubs['Subscription']=='Docurama','2020-04':]=TotMSubs.loc[TotMSubs['Subscription']=='Docurama','2020-04':].div(4.99)
TotMSubs.loc[TotMSubs['Subscription']=='Dove','2020-04':]=TotMSubs.loc[TotMSubs['Subscription']=='Dove','2020-04':].div(4.99)
TotMSubs.loc[TotMSubs['Subscription']=='Fandor','2020-04':]=TotMSubs.loc[TotMSubs['Subscription']=='Fandor','2020-04':].div(3.99)
TotMSubs.loc[TotMSubs['Subscription']=='Screambox','2020-04':]=TotMSubs.loc[TotMSubs['Subscription']=='Screambox','2020-04':].div(4.99)

In [158]:
SubP= TotMSubs.set_index('Subscription').T
SubP.reset_index(inplace=True)
SubP.columns=['Date','CONtv','Docurama','Dove','Fandor','Screambox','Number']

In [159]:
SubPrj= pd.DataFrame([
    ['CONtv',linear_regression_int(SubP['Number'],SubP['CONtv']),linear_regression_slope(SubP['Number'],SubP['CONtv'])],
    ['Docurama',linear_regression_int(SubP['Number'],SubP['Docurama']),linear_regression_slope(SubP['Number'],SubP['Docurama'])],
    ['Dove',linear_regression_int(SubP['Number'],SubP['Dove']),linear_regression_slope(SubP['Number'],SubP['Dove'])],
    ['Fandor',linear_regression_int(SubP['Number'],SubP['Fandor']),linear_regression_slope(SubP['Number'],SubP['Fandor'])],
    ['Screambox',linear_regression_int(SubP['Number'],SubP['Screambox']),linear_regression_slope(SubP['Number'],SubP['Screambox'])]
    
], 
                     columns=['Subscription','intercept','Slope',])

In [160]:
SubP

,Date,CONtv,Docurama,Dove,Fandor,Screambox,Number
0,2020-04,27348.0,10463.0,26531.0,44745.0,5955.0,1.0
1,2020-05,29976.0,11374.0,29572.0,48415.0,7830.0,2.0
2,2020-06,35153.0,13044.0,33655.0,56256.0,9596.0,3.0
3,2020-07,37420.0,13664.0,34869.0,56155.0,9884.0,4.0
4,2020-08,37374.0,13893.0,36957.0,55747.0,9766.0,5.0
5,2020-09,36964.0,14875.0,37103.0,55683.0,9906.0,6.0
6,2020-10,37477.0,15981.0,37610.0,56846.0,11052.0,7.0
7,2020-11,38766.0,15990.0,38076.0,57636.0,10837.0,8.0
8,2020-12,38847.0,15986.0,38909.0,58107.0,11815.0,9.0
9,2021-01,39384.0,16298.0,39306.0,59199.0,13748.0,10.0


In [161]:
SubPrj

,Subscription,intercept,Slope
0,CONtv,32749.990476,1107.387902
1,Docurama,8958.450794,957.086744
2,Dove,28243.857143,1138.187902
3,Fandor,49459.369841,1178.100129
4,Screambox,-33849.696825,4370.238867


In [162]:
TotMSubs

,Subscription,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,CONtv,27348.0,29976.0,35153.0,37420.0,37374.0,36964.0,37477.0,38766.0,38847.0,39384.0,40260.0,41580.0,45140.0,47190.0,48426.0,50442.0,51438.0,66621.0,67433.0,64338.0,63598.0,65772.0,65728.0,65804.0,67317.0,68751.0,68908.0,69543.0,69843.0,67076.0,62936.0,60942.0,59793.0,59611.0,59713.0,59608.0
1,Docurama,10463.0,11374.0,13044.0,13664.0,13893.0,14875.0,15981.0,15990.0,15986.0,16298.0,16546.0,17024.0,17691.0,19451.0,20590.0,22303.0,24635.0,31447.0,31794.0,31236.0,31156.0,31796.0,32726.0,34161.0,35287.0,35992.0,36704.0,38034.0,38826.0,38477.0,37545.0,38001.0,38480.0,39371.0,39325.0,39758.0
2,Dove,26531.0,29572.0,33655.0,34869.0,36957.0,37103.0,37610.0,38076.0,38909.0,39306.0,39695.0,39826.0,40557.0,41391.0,41673.0,42469.0,43493.0,50179.0,50795.0,52151.0,57031.0,59256.0,57811.0,56932.0,58074.0,57829.0,58135.0,57690.0,57898.0,59576.0,60229.0,63534.0,65831.0,69030.0,69984.0,71155.0
3,Fandor,44745.0,48415.0,56256.0,56155.0,55747.0,55683.0,56846.0,57636.0,58107.0,59199.0,59925.0,60891.0,62280.0,63736.0,64435.0,65923.0,66957.0,80373.0,80336.0,79567.0,79115.0,81684.0,81186.0,81430.0,82759.0,82333.0,82134.0,83069.0,84466.0,83609.0,82692.0,83762.0,84403.0,86805.0,86620.0,85873.0
4,Screambox,5955.0,7830.0,9596.0,9884.0,9766.0,9906.0,11052.0,10837.0,11815.0,13748.0,13897.0,13760.0,14101.0,14622.0,15091.0,16188.0,16915.0,23254.0,26160.0,28649.0,28257.0,30082.0,32379.0,35896.0,37562.0,36820.0,36312.0,36268.0,45621.0,51672.0,62311.0,183310.0,220086.0,198780.0,189111.0,184497.0
-1,Number,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0


# Subs Projections (Last 6 Months)

In [163]:
SubP6=SubP.iloc[-6:]

In [164]:
Sub6Prj= pd.DataFrame([
    ['CONtv',linear_regression_int(SubP6['Number'],SubP6['CONtv']),linear_regression_slope(SubP6['Number'],SubP6['CONtv'])],
    ['Docurama',linear_regression_int(SubP6['Number'],SubP6['Docurama']),linear_regression_slope(SubP6['Number'],SubP6['Docurama'])],
    ['Dove',linear_regression_int(SubP6['Number'],SubP6['Dove']),linear_regression_slope(SubP6['Number'],SubP6['Dove'])],
    ['Fandor',linear_regression_int(SubP6['Number'],SubP6['Fandor']),linear_regression_slope(SubP6['Number'],SubP6['Fandor'])],
    ['Screambox',linear_regression_int(SubP6['Number'],SubP6['Screambox']),linear_regression_slope(SubP6['Number'],SubP6['Screambox'])]
    
], 
                     columns=['Subscription','intercept','Slope',])

In [165]:
Sub6Prj

,Subscription,intercept,Slope
0,CONtv,80063.876190,-585.971429
1,Docurama,23501.295238,455.085714
2,Dove,-7244.161905,2205.114286
3,Fandor,59296.876190,768.028571
4,Screambox,-407995.723810,17343.628571


In [166]:
SubP6

,Date,CONtv,Docurama,Dove,Fandor,Screambox,Number
30,2022-10,62936.0,37545.0,60229.0,82692.0,62311.0,31.0
31,2022-11,60942.0,38001.0,63534.0,83762.0,183310.0,32.0
32,2022-12,59793.0,38480.0,65831.0,84403.0,220086.0,33.0
33,2023-01,59611.0,39371.0,69030.0,86805.0,198780.0,34.0
34,2023-02,59713.0,39325.0,69984.0,86620.0,189111.0,35.0
35,2023-03,59608.0,39758.0,71155.0,85873.0,184497.0,36.0


# Percent Rev increase by Year

In [167]:
PercentSub= pd.DataFrame([
    #Year 1
    [ProjTable.iloc[3]['Number'], ProjTable.iloc[3]['Date']+'_+1Year_(LinReg)',   (ProjTable.iloc[3]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[3]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[3]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[3]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[3]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    #Year 2 
    [ProjTable.iloc[4]['Number'], ProjTable.iloc[4]['Date']+'_+2Year_(LinReg)',   (ProjTable.iloc[4]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[4]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[4]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[4]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[4]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    #Year 3
    [ProjTable.iloc[5]['Number'], ProjTable.iloc[5]['Date']+'_+3Year_(LinReg)',   (ProjTable.iloc[5]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[5]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[5]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[5]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[5]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    #Year 4
    [ProjTable.iloc[6]['Number'], ProjTable.iloc[6]['Date']+'_+4Year_(LinReg)',   (ProjTable.iloc[6]['CONtv']-ProjTable.iloc[1]['CONtv'])/ProjTable.iloc[1]['CONtv'],
                                                                         (ProjTable.iloc[6]['Docurama']-ProjTable.iloc[1]['Docurama'])/ProjTable.iloc[1]['Docurama'],
                                                                         (ProjTable.iloc[6]['Dove']-ProjTable.iloc[1]['Dove'])/ProjTable.iloc[1]['Dove'],
                                                                         (ProjTable.iloc[6]['Fandor']-ProjTable.iloc[1]['Fandor'])/ProjTable.iloc[1]['Fandor'],
                                                                         (ProjTable.iloc[6]['Screambox']-ProjTable.iloc[1]['Screambox'])/ProjTable.iloc[1]['Screambox'] ],
    
], 
    columns=['Number','Date','CONtv','Docurama','Dove','Fandor','Screambox'])

In [168]:
PercentSub= PercentSub.loc[:,('Date','CONtv','Docurama','Dove','Fandor','Screambox')]
PercentSub= PercentSub.set_index('Date').T


In [169]:
PercentSub

Date,4-2023_+1Year_(LinReg),4-2024_+2Year_(LinReg),4-2025_+3Year_(LinReg),4-2026_+4Year_(LinReg)
CONtv,0.441159,0.664093,0.887027,1.109961
Docurama,0.380819,0.669693,0.958567,1.247440
Dove,0.164737,0.356688,0.548639,0.740589
Fandor,0.234476,0.399105,0.563735,0.728364
Screambox,-0.046479,0.237769,0.522017,0.806264


In [170]:
PercentSub.reset_index(inplace=True)

In [171]:
PercentSub.columns=['Subscription','+1Year%_(LinReg)','+2Year%_(LinReg)','+3Year%_(LinReg)','+4Year%_(LinReg)']

In [172]:
PercentSub

,Subscription,+1Year%_(LinReg),+2Year%_(LinReg),+3Year%_(LinReg),+4Year%_(LinReg)
0,CONtv,0.441159,0.664093,0.887027,1.109961
1,Docurama,0.380819,0.669693,0.958567,1.247440
2,Dove,0.164737,0.356688,0.548639,0.740589
3,Fandor,0.234476,0.399105,0.563735,0.728364
4,Screambox,-0.046479,0.237769,0.522017,0.806264


In [173]:
Rev1['Yearly Avg']= Rev1['All Platforms monthly Rev Avg (10/8 estimate)']*12

In [174]:
Rev1=pd.merge(Rev1,PercentSub, on='Subscription', how='left')

In [175]:
Rev1['+1 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+1Year%_(LinReg)'])
Rev1['+2 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+2Year%_(LinReg)'])
Rev1['+3 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+3Year%_(LinReg)'])
Rev1['+4 Year Proj.']= Rev1['Yearly Avg']* (1+ Rev1['+4Year%_(LinReg)'])

#New Rev
Rev1['+1 Year Proj. (New Rev)']= Rev1['+1 Year Proj.'] - Rev1['Yearly Avg']
Rev1['+2 Year Proj. (New Rev)']= Rev1['+2 Year Proj.'] - Rev1['+1 Year Proj.']
Rev1['+3 Year Proj. (New Rev)']= Rev1['+3 Year Proj.'] - Rev1['+2 Year Proj.']
Rev1['+4 Year Proj. (New Rev)']= Rev1['+4 Year Proj.'] - Rev1['+3 Year Proj.']

In [176]:
Rev1

,Subscription,ID,TV/MOVIE,Title,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Roku+Amzn monthly Rev Avg,Trend,All Platforms monthly Rev Avg (10/8 estimate),Yearly Avg,+1Year%_(LinReg),+2Year%_(LinReg),+3Year%_(LinReg),+4Year%_(LinReg),+1 Year Proj.,+2 Year Proj.,+3 Year Proj.,+4 Year Proj.,+1 Year Proj. (New Rev),+2 Year Proj. (New Rev),+3 Year Proj. (New Rev),+4 Year Proj. (New Rev)
0,CONtv,1000000000019,MOVIE,The Astrologer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014874,35.507753,31.604657,23.711389,21.538645,15.009008,1.755791,0.490172,1.279526,2.536042,4.645040,4.929616,0.041680,0.837066,3.966909,4.184444,6.115627,9.304014,0.381297,11.630017,139.560210,0.441159,0.664093,0.887027,1.109961,201.128462,232.241190,263.353917,294.466644,61.568252,31.112727,31.112727,31.112727
1,CONtv,1000000000029,MOVIE,Psychos In Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065069,0.046843,5.991302,13.181775,9.624143,16.630251,12.796687,5.492900,1.042132,1.217194,1.485035,3.702427,4.204093,9.484689,1.553692,4.031319,2.890775,13.347167,7.946219,6.038616,0.437842,7.548271,90.579246,0.441159,0.664093,0.887027,1.109961,130.539102,150.732303,170.925504,191.118706,39.959856,20.193201,20.193201,20.193201
2,CONtv,1000000000180,MOVIE,Guardian Of The Highlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022621,4.319257,1.541264,0.191503,0.036198,2.647592,NaN,1.154242,0.033705,NaN,0.546534,0.057599,2.811503,5.696378,6.934644,0.359978,1.546786,3.065467,4.812656,4.530254,2.239343,0.265155,2.799179,33.590151,0.441159,0.664093,0.887027,1.109961,48.408751,55.897141,63.385531,70.873920,14.818600,7.488390,7.488390,7.488390
3,CONtv,1000000000196,MOVIE,The Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028075,6.071482,8.630787,6.161522,1.751147,0.124139,0.509566,0.032920,0.202122,0.676859,0.002097,0.127751,0.593716,0.642548,2.325786,2.757730,1.300248,1.878735,0.079511,2.348419,28.181024,0.441159,0.664093,0.887027,1.109961,40.613338,46.895848,53.178359,59.460870,12.432314,6.282511,6.282511,6.282511
4,CONtv,1000000000197,MOVIE,Bad Man's River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130151,10.490839,23.987585,9.109721,9.593824,6.842495,9.670156,5.142304,0.555129,1.405560,1.488159,6.806758,16.976588,2.056589,0.657016,3.276972,9.857469,8.960031,21.939094,7.839286,1.211608,9.799108,117.589297,0.441159,0.664093,0.887027,1.109961,169.464881,195.679543,221.894205,248.108867,51.875585,26.214662,26.214662,26.214662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,Screambox,WELCOMETODAISYLAND,TV,Welcome to Daisyland,32.395970,20.035365,19.292995,22.000019,10.858643,5.286587,20.26157,5.830574,5.72241,8.342684,2.81162,5.165193,13.859854,6.321581,7.919916,25.306872,21.557545,11.054143,17.392220,25.040735,2.163035,1.553928,2.235634,2.660140,0.789280,2.040818,1.400696,6.676877,3.942059,1.602328,5.586020,9.031688,14.274350,19.659808,9.379428,15.285866,10.687179,-0.488860,13.358974,160.307686,-0.046479,0.237769,0.522017,0.806264,152.856749,198.423856,243.990964,289.558071,-7.450937,45.567107,45.567107,45.567107
9574,Screambox,WEREWOLFFEVER,MOVIE,Werewolf Fever,131.201122,110.220321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.710721,-20.980801,150.888402,1810.660821,-0.046479,0.237769,0.522017,0.806264,1726.503165,2241.179521,2755.855876,3270.532232,-84.157656,514.676356,514.676356,514.676356
9575,Screambox,WITHOUTNAME

In [177]:
Content2= Content2.loc[:,('Subscription','Year','Date','Month','Month Abv','ID','TV/MOVIE','Title',
                          'Amzn Min Streamed','Amzn Streams','Amzn Min/Stream','Roku Min Streamed','Roku Streams','Roku Min/Stream',
                          'Total Min Streamed','Percentile Rank (Min)','Total Streams','Min/Stream','Content Month Rev')]

# Last Look

In [178]:
Subs.head(10)

,Subscription,Year,Month(num),Month,Billing Amount,Amazon Paid Subs,Amzn Rev,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Amzn Min/Sub,Roku Paid Subs,Roku Rev,Roku Min Streamed,Roku Streams,Roku Min/Stream,Roku Min/Sub,Total Paid Subs,Revenue,Total Min Streamed,Total Streams,Avg Min/Stream,Min/Sub
0,CONtv,2020,4,Apr,4.99,26140,130438.60,2494913.23,111911.0,22.293727,95.444270,1208,6027.92,1.743553e+07,675973.0,25.793239,14433.388537,27348,136466.52,1.993045e+07,787884.0,25.296169,728.771632
1,CONtv,2020,5,May,4.99,27697,138208.03,2569089.38,115117.0,22.317202,92.756955,2279,11372.21,1.957893e+07,689901.0,28.379336,8591.019015,29976,149580.24,2.214802e+07,805018.0,27.512455,738.858477
2,CONtv,2020,6,Jun,4.99,32540,162374.60,2297150.60,101963.0,22.529257,70.594671,2613,13038.87,1.380414e+07,468805.0,29.445386,5282.871892,35153,175413.47,1.610129e+07,570768.0,28.209877,458.034730
3,CONtv,2020,7,Jul,4.99,34989,174595.11,2832018.33,117724.0,24.056423,80.940248,2431,12130.69,1.215672e+07,412014.0,29.505606,5000.708696,37420,186725.80,1.498874e+07,529738.0,28.294631,400.554280
4,CONtv,2020,8,Aug,4.99,35142,175358.58,2205753.84,96344.0,22.894564,62.766884,2232,11137.68,1.070387e+07,371194.0,28.836332,4795.642175,37374,186496.26,1.290963e+07,467538.0,27.611931,345.417327
5,CONtv,2020,9,Sep,4.99,34955,174425.45,2164008.28,92631.0,23.361599,61.908405,2009,10024.91,1.307546e+07,451543.0,28.957281,6508.440839,36964,184450.36,1.523947e+07,544174.0,28.004767,412.278593
6,CONtv,2020,10,Oct,4.99,35633,177808.67,2365955.76,96221.0,24.588767,66.397883,1844,9201.56,1.150928e+07,390161.0,29.498808,6241.477511,37477,187010.23,1.387524e+07,486382.0,28.527454,370.233484
7,CONtv,2020,11,Nov,4.99,36933,184295.67,2832501.78,128918.0,21.971344,76.692979,1833,9146.67,1.093405e+07,366013.0,29.873380,5965.109269,38766,193442.34,1.376655e+07,494931.0,27.815083,355.119101
8,CONtv,2020,12,Dec,4.99,36900,184131.00,2269143.46,93255.0,24.332673,61.494403,1947,9715.53,8.677735e+06,296125.0,29.304297,4456.977437,38847,193846.53,1.094688e+07,389380.0,28.113613,281.794695
9,CONtv,2021,1,Jan,4.99,37155,185403.45,2377579.88,94629.0,25.125277,63.990846,2229,11122.71,7.482066e+06,304418.0,24.578263,3356.691583,39384,196526.16,9.859645e+06,399047.0,24.707980,250.346471


In [179]:
Content2.loc[Content2['Title'].str.contains(': Sammy', case=False)]

,Subscription,Year,Date,Month,Month Abv,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Roku Min Streamed,Roku Streams,Roku Min/Stream,Total Min Streamed,Percentile Rank (Min),Total Streams,Min/Stream,Content Month Rev
1173,Dove,2020,2020-04,04,Apr,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,37515.08,947.0,39.614657,1807.6662,31.0,58.311813,39322.7462,0.956600,978.0,40.207307,51.653978
5405,Dove,2020,2020-05,05,May,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,37153.53,898.0,41.373641,4064.4996,63.0,64.515867,41218.0296,0.960289,961.0,42.890770,61.581900
9542,Dove,2020,2020-06,06,Jun,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,30305.14,797.0,38.024015,3696.4998,69.0,53.572461,34001.6398,0.963100,866.0,39.262864,66.839217
13461,Dove,2020,2020-07,07,Jul,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,41854.22,1060.0,39.485113,2160.1662,40.0,54.004155,44014.3862,0.966543,1100.0,40.013078,93.403402
16992,Dove,2020,2020-08,08,Aug,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,30931.01,741.0,41.742254,3820.8330,86.0,44.428291,34751.8430,0.960894,827.0,42.021576,79.554577
20357,Dove,2020,2020-09,09,Sep,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,23752.24,508.0,46.756378,2731.1664,46.0,59.373183,26483.4064,0.957328,554.0,47.803983,65.544666
23775,Dove,2020,2020-10,10,Oct,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,32796.57,763.0,42.983709,3282.3330,62.0,52.940855,36078.9030,0.966535,825.0,43.732004,107.152569
27192,Dove,2020,2020-11,11,Nov,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,21919.82,576.0,38.055243,1302.4998,24.0,54.270825,23222.3198,0.945180,600.0,38.703866,75.269969
30791,Dove,2020,2020-12,12,Dec,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,20158.81,517.0,38.991896,977.6664,25.0,39.106656,21136.4764,0.929889,542.0,38.997189,93.184743
34556,Dove,2021,2021-01,01,Jan,883476075594,MOVIE,A Turtle's Tale: Sammy's Adventures,32034.26,728.0,44.003104,1784.8332,35.0,50.995234,33819.0932,0.963028,763.0,44.323844,117.065389


In [180]:
Content2

,Subscription,Year,Date,Month,Month Abv,ID,TV/MOVIE,Title,Amzn Min Streamed,Amzn Streams,Amzn Min/Stream,Roku Min Streamed,Roku Streams,Roku Min/Stream,Total Min Streamed,Percentile Rank (Min),Total Streams,Min/Stream,Content Month Rev
0,CONtv,2020,2020-04,04,Apr,CEG10018117,MOVIE,100 Below Zero,89.02,2.0,44.510000,1015.9998,18.0,56.444433,1105.0198,0.614765,20.0,55.250990,7.566223
1,CONtv,2020,2020-04,04,Apr,CEG10018177,MOVIE,100 Ghost Street: The Return of Richard Speck,1.08,1.0,1.080000,845.4996,15.0,56.366640,846.5796,0.577181,16.0,52.911225,5.796647
2,CONtv,2020,2020-04,04,Apr,CEG10018118,MOVIE,100 Million BC,1016.65,19.0,53.507895,1116.3330,26.0,42.935885,2132.9830,0.718121,45.0,47.399622,14.604829
3,CONtv,2020,2020-04,04,Apr,CEG10018116,MOVIE,11/11/11,292.05,7.0,41.721429,423.0000,9.0,47.000000,715.0500,0.551678,16.0,44.690625,4.896046
4,CONtv,2020,2020-04,04,Apr,CEG10020241,MOVIE,13 Demons,716.62,23.0,31.157391,0.0000,0.0,NaN,716.6200,0.553020,23.0,31.157391,4.906796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165710,Screambox,2023,2023-03,03,Mar,THEDEADZONE,TV,The Dead Zone,10868.09,394.0,27.583985,452286.0264,12584.0,35.941356,463154.1164,0.995402,12978.0,35.687634,16350.410936
165711,Screambox,2023,2023-03,03,Mar,THEISLAND,TV,The Island,45435.87,1651.0,27.520212,140363.9496,4662.0,30.108097,185799.8196,0.989655,6313.0,29.431304,6559.163127
165712,Screambox,2023,2023-03,03,Mar,TOXICCRUSADERS,TV,Toxic Crusaders,7324.44,480.0,15.259250,11615.4558,819.0,14.182486,18939.8958,0.862069,1299.0,14.580366,668.622103
165713,Screambox,2023,2023-03,03,Mar,WELCOMETODAISYLAND,TV,Welcome to Daisyland,0.00,0.0,NaN,432.9990,90.0,4.811100,432.9990,0.254023,90.0,4.811100,15.285866


In [181]:
CM

,Subscription,ID,TV/MOVIE,Title,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Metric,Average,Trend
0,CONtv,1000000000019,MOVIE,The Astrologer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.800000,1634.420000,1883.400000,1513.960000,1453.460000,1082.100000,654.71000,251.340000,729.080000,354.82000,332.250000,237.700000,11.810000,106.420000,337.440000,248.360000,492.120000,Minutes Streamed,666.128824,30.707500
1,CONtv,1000000000029,MOVIE,Psychos In Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.166200,2.666400,322.243200,606.756400,573.526600,1061.833000,863.539600,396.020000,388.59660,624.126600,846.180000,518.01000,300.710000,457.340000,440.240000,512.520000,245.900000,792.196600,639.426400,Minutes Streamed,504.999874,35.347789
2,CONtv,1000000000180,MOVIE,Guardian Of The Highlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666200,210.169800,87.732800,10.300000,1.666200,157.776600,NaN,77.890000,2.430000,NaN,280.240000,32.820000,393.36000,407.450000,334.380000,102.000000,196.650000,260.760000,285.646400,364.546200,Minutes Streamed,178.193567,21.345882
3,CONtv,1000000000196,MOVIE,The Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.510000,279.470000,514.330000,393.410000,118.170000,8.950000,190.01000,16.880000,115.170000,94.70000,0.150000,6.160000,168.230000,81.690000,197.840000,163.680000,104.630000,Minutes Streamed,144.410588,6.445000
4,CONtv,1000000000197,MOVIE,Bad Man's River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333000,597.166200,1290.176400,419.320000,571.719800,436.889800,652.556600,370.743200,207.00000,720.712800,847.960000,952.33980,1214.299800,99.166200,186.166200,416.616400,838.512800,531.806200,1765.422800,Minutes Streamed,638.153053,97.727211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,Screambox,WELCOMETODAISYLAND,TV,Welcome to Daisyland,0.542056,0.464455,0.463918,0.578125,0.497326,0.22,0.388889,0.255102,0.349754,0.319048,0.129187,0.336538,0.242857,0.194444,0.284404,0.30315,0.372881,0.274667,0.260417,0.324561,0.240741,0.159204,0.191693,0.214506,0.081897,0.21733,0.198034,0.339943,0.31044,0.154967,0.340702,0.336761,0.291415,0.318236,0.174432,0.254023,Percentile Rank (Min),0.295169,-0.008230
9574,Screambox,WEREWOLFFEVER,MOVIE,Werewolf Fever,0.827103,0.848341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentile Rank (Min),0.837722,0.021238
9575,Screambox,WITHOUTNAME,MOVIE,Without Name,0.845794,0.739336,0.680412,0.890625,0.871658,0.78,0.661616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentile Rank (Min),0.781349,-0.030696
9576,Screambox,YOURESOCOOLBREWSTER,TV,"You're So Cool, Brewster! The Story of Fright ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.594278,0.485861,0.540508,0.466031,0.695341,0.803448,Percentile Rank (Min),0.597578,0.041834


In [182]:
SubP

,Date,CONtv,Docurama,Dove,Fandor,Screambox,Number
0,2020-04,27348.0,10463.0,26531.0,44745.0,5955.0,1.0
1,2020-05,29976.0,11374.0,29572.0,48415.0,7830.0,2.0
2,2020-06,35153.0,13044.0,33655.0,56256.0,9596.0,3.0
3,2020-07,37420.0,13664.0,34869.0,56155.0,9884.0,4.0
4,2020-08,37374.0,13893.0,36957.0,55747.0,9766.0,5.0
5,2020-09,36964.0,14875.0,37103.0,55683.0,9906.0,6.0
6,2020-10,37477.0,15981.0,37610.0,56846.0,11052.0,7.0
7,2020-11,38766.0,15990.0,38076.0,57636.0,10837.0,8.0
8,2020-12,38847.0,15986.0,38909.0,58107.0,11815.0,9.0
9,2021-01,39384.0,16298.0,39306.0,59199.0,13748.0,10.0


In [183]:
SubPrj

,Subscription,intercept,Slope
0,CONtv,32749.990476,1107.387902
1,Docurama,8958.450794,957.086744
2,Dove,28243.857143,1138.187902
3,Fandor,49459.369841,1178.100129
4,Screambox,-33849.696825,4370.238867


In [184]:
SubP6

,Date,CONtv,Docurama,Dove,Fandor,Screambox,Number
30,2022-10,62936.0,37545.0,60229.0,82692.0,62311.0,31.0
31,2022-11,60942.0,38001.0,63534.0,83762.0,183310.0,32.0
32,2022-12,59793.0,38480.0,65831.0,84403.0,220086.0,33.0
33,2023-01,59611.0,39371.0,69030.0,86805.0,198780.0,34.0
34,2023-02,59713.0,39325.0,69984.0,86620.0,189111.0,35.0
35,2023-03,59608.0,39758.0,71155.0,85873.0,184497.0,36.0


In [185]:
Sub6Prj

,Subscription,intercept,Slope
0,CONtv,80063.876190,-585.971429
1,Docurama,23501.295238,455.085714
2,Dove,-7244.161905,2205.114286
3,Fandor,59296.876190,768.028571
4,Screambox,-407995.723810,17343.628571


In [186]:
Rev1

,Subscription,ID,TV/MOVIE,Title,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,Roku+Amzn monthly Rev Avg,Trend,All Platforms monthly Rev Avg (10/8 estimate),Yearly Avg,+1Year%_(LinReg),+2Year%_(LinReg),+3Year%_(LinReg),+4Year%_(LinReg),+1 Year Proj.,+2 Year Proj.,+3 Year Proj.,+4 Year Proj.,+1 Year Proj. (New Rev),+2 Year Proj. (New Rev),+3 Year Proj. (New Rev),+4 Year Proj. (New Rev)
0,CONtv,1000000000019,MOVIE,The Astrologer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014874,35.507753,31.604657,23.711389,21.538645,15.009008,1.755791,0.490172,1.279526,2.536042,4.645040,4.929616,0.041680,0.837066,3.966909,4.184444,6.115627,9.304014,0.381297,11.630017,139.560210,0.441159,0.664093,0.887027,1.109961,201.128462,232.241190,263.353917,294.466644,61.568252,31.112727,31.112727,31.112727
1,CONtv,1000000000029,MOVIE,Psychos In Love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065069,0.046843,5.991302,13.181775,9.624143,16.630251,12.796687,5.492900,1.042132,1.217194,1.485035,3.702427,4.204093,9.484689,1.553692,4.031319,2.890775,13.347167,7.946219,6.038616,0.437842,7.548271,90.579246,0.441159,0.664093,0.887027,1.109961,130.539102,150.732303,170.925504,191.118706,39.959856,20.193201,20.193201,20.193201
2,CONtv,1000000000180,MOVIE,Guardian Of The Highlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022621,4.319257,1.541264,0.191503,0.036198,2.647592,NaN,1.154242,0.033705,NaN,0.546534,0.057599,2.811503,5.696378,6.934644,0.359978,1.546786,3.065467,4.812656,4.530254,2.239343,0.265155,2.799179,33.590151,0.441159,0.664093,0.887027,1.109961,48.408751,55.897141,63.385531,70.873920,14.818600,7.488390,7.488390,7.488390
3,CONtv,1000000000196,MOVIE,The Baby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028075,6.071482,8.630787,6.161522,1.751147,0.124139,0.509566,0.032920,0.202122,0.676859,0.002097,0.127751,0.593716,0.642548,2.325786,2.757730,1.300248,1.878735,0.079511,2.348419,28.181024,0.441159,0.664093,0.887027,1.109961,40.613338,46.895848,53.178359,59.460870,12.432314,6.282511,6.282511,6.282511
4,CONtv,1000000000197,MOVIE,Bad Man's River,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130151,10.490839,23.987585,9.109721,9.593824,6.842495,9.670156,5.142304,0.555129,1.405560,1.488159,6.806758,16.976588,2.056589,0.657016,3.276972,9.857469,8.960031,21.939094,7.839286,1.211608,9.799108,117.589297,0.441159,0.664093,0.887027,1.109961,169.464881,195.679543,221.894205,248.108867,51.875585,26.214662,26.214662,26.214662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,Screambox,WELCOMETODAISYLAND,TV,Welcome to Daisyland,32.395970,20.035365,19.292995,22.000019,10.858643,5.286587,20.26157,5.830574,5.72241,8.342684,2.81162,5.165193,13.859854,6.321581,7.919916,25.306872,21.557545,11.054143,17.392220,25.040735,2.163035,1.553928,2.235634,2.660140,0.789280,2.040818,1.400696,6.676877,3.942059,1.602328,5.586020,9.031688,14.274350,19.659808,9.379428,15.285866,10.687179,-0.488860,13.358974,160.307686,-0.046479,0.237769,0.522017,0.806264,152.856749,198.423856,243.990964,289.558071,-7.450937,45.567107,45.567107,45.567107
9574,Screambox,WEREWOLFFEVER,MOVIE,Werewolf Fever,131.201122,110.220321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.710721,-20.980801,150.888402,1810.660821,-0.046479,0.237769,0.522017,0.806264,1726.503165,2241.179521,2755.855876,3270.532232,-84.157656,514.676356,514.676356,514.676356
9575,Screambox,WITHOUTNAME

In [187]:
output= pd.ExcelWriter('Amazon_Roku SVOD '+Monthsinc+' out.xlsx', engine='xlsxwriter')

Subs.to_excel(output, sheet_name= 'Subscriptions',index=False)
Content2.to_excel(output, sheet_name= 'Content',index=False)
CM.to_excel(output, sheet_name='Monthly Title Trend', index=False)
# SubP.to_excel(output, sheet_name='Sub Projections', index=False)
# SubPrj.to_excel(output, sheet_name='Sub Slp_int', index=False)
# SubP6.to_excel(output, sheet_name='Sub6 Projections', index=False)
# Sub6Prj.to_excel(output, sheet_name='Sub6 Slp_int', index=False)
Rev1.to_excel(output, sheet_name='Revenue', index=False)

output.save()